In [1]:
import numpy as np
import pandas as pd
from statistics import mean
from nltk.tokenize import word_tokenize
from datetime import datetime, date, timedelta  
pd.set_option('display.max_columns', None)

In [2]:
tennis = pd.read_csv('data/tennis.csv')

C:\Users\Federico\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#trasformo tutti i valori riferiti al livello in str, altrimenti 
#mi da dei problemi perchè alcuni valori vengono letti come stringa ed altri come interi
tennis=tennis.astype({'tourney_level':str})

In [4]:
tennis

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
0,2019-M020,Brisbane,Hard,32,A,20181231,300,105453,NaN,Kei Nishikori,R,178.0,JPN,29.004791,106421,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,6-4 3-6 6-2,3,F,124.0,3.0,3.0,77.0,44.0,31.0,17.0,13.0,3.0,6.0,8.0,6.0,100.0,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0,3928,742618.69
1,2019-M020,Brisbane,Hard,32,A,20181231,299,106421,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,104542,PR,Jo-Wilfried Tsonga,R,188.0,FRA,33.705681,7-6(6) 6-2,3,SF,82.0,10.0,1.0,52.0,33.0,28.0,14.0,10.0,0.0,1.0,17.0,2.0,77.0,52.0,36.0,7.0,10.0,10.0,13.0,16.0,1977.0,239.0,200.0,3928,742618.69
2,2019-M020,Brisbane,Hard,32,A,20181231,298,105453,NaN,Kei Nishikori,R,178.0,JPN,29.004791,104871,NaN,Jeremy Chardy,R,188.0,FRA,31.882272,6-2 6-2,3,SF,66.0,2.0,2.0,47.0,33.0,26.0,9.0,8.0,2.0,2.0,10.0,3.0,46.0,27.0,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0,3928,742618.69
3,2019-M020,Brisbane,Hard,32,A,20181231,297,104542,PR,Jo-Wilfried Tsonga,R,188.0,FRA,33.705681,200282,NaN,Alex De Minaur,R,183.0,AUS,19.868583,6-4 7-6(2),3,QF,106.0,12.0,2.0,68.0,43.0,34.0,15.0,11.0,4.0,5.0,1.0,2.0,81.0,60.0,38.0,9.0,11.0,4.0,6.0,239.0,200.0,31.0,1298.0,3928,742618.69
4,2019-M020,Brisbane,Hard,32,A,20181231,296,106421,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,105683,NaN,Milos Raonic,R,196.0,CAN,28.010951,6-7(2) 6-3 6-4,3,QF,129.0,12.0,3.0,105.0,68.0,48.0,25.0,16.0,8.0,8.0,29.0,5.0,94.0,56.0,46.0,19.0,15.0,2.0,4.0,16.0,1977.0,18.0,1855.0,3928,742618.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186068,2020-W-ITF-USA-48A-2020,W25 Orlando FL,Hard,32,25,20201109,204,215794,WC,Hurricane Tyra Black,R,NaN,USA,19.690623,222081,NaN,Eleana Yu,R,NaN,USA,16.002738,7-6(1) 6-1,3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,598.0,57.0,1216.0,6.0,2312,471620.95
186069,2020-W-ITF-USA-48A-2020,W25 Orlando FL,Hard,32,25,20201109,205,210754,NaN,Vivian Heisen,U,NaN,GER,26.869268,220723,NaN,Jessica Livianu,R,NaN,USA,22.713210,2-6 6-2 10-3,3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,429.0,102.0,729.0,35.0,2312,471620.95
186070,2020-W-ITF-USA-48A-2020,W25 Orlando FL,Hard,32,25,20201109,206,211874,NaN,Victoria Rodriguez,U,NaN,MEX,25.552361,202478,NaN,Tara Moore,R,NaN,GBR,28.260096,6-2 7-6(8),3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,643.0,48.0,454.0,93.0,2312,471620.95
186071,2020-W-ITF-USA-48A-2020,W25 Orlando FL,Hard,32,25,20201109,207,213789,NaN,Katarina Jokic,U,NaN,BIH,22.559890,205922,NaN,Chanelle Van Nguyen,U,NaN,USA,26.806297,6-4 6-2,3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,923.0,17.0,627.0,51.0,2312,471620.95


In [5]:
#elimino le row duplicate
tennis=tennis.drop_duplicates(keep='first')

In [6]:
tennis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185764 entries, 0 to 186072
Data columns (total 49 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          185764 non-null  object 
 1   tourney_name        185764 non-null  object 
 2   surface             185602 non-null  object 
 3   draw_size           185764 non-null  int64  
 4   tourney_level       185764 non-null  object 
 5   tourney_date        185764 non-null  int64  
 6   match_num           185764 non-null  int64  
 7   winner_id           185764 non-null  int64  
 8   winner_entry        25806 non-null   object 
 9   winner_name         185764 non-null  object 
 10  winner_hand         185748 non-null  object 
 11  winner_ht           49290 non-null   float64
 12  winner_ioc          185764 non-null  object 
 13  winner_age          182936 non-null  float64
 14  loser_id            185764 non-null  int64  
 15  loser_entry         44080 non-null

In [7]:
#per riempire successivamente alcuni missing value, utilizzando medie dei valori in base al sesso,
# mi servono le due tabella male_players e female_players

male = pd.read_csv('data/male_players.csv')
female = pd.read_csv('data/female_players.csv')
#elimino le righe dove sono presenti null value su nome o cognome
male=male.dropna()
female=female.dropna()

#creo un set per male e uno per female in modo da poter poi fare ricerche hashing rapide. Concateno nome e cognome insieme
#stando attento alla spaziatura
male_set=set()
for index, row in male.iterrows():

    value=''
    for val in row:
        value=value + val +' '
    value=value[:-1]
    male_set.add(value)
    
    
female_set=set()

for index, row in female.iterrows():
    value=''
    for val in row:
        value=value + val +' '
    value=value[:-1]
    female_set.add(value)


In [8]:
male

,name,surname
0,Gardnar,Mulloy
1,Pancho,Segura
2,Frank,Sedgman
3,Giuseppe,Merlo
4,Richard Pancho,Gonzales
...,...,...
55203,Borys,Zgola
55204,Noah,Schlagenhauf
55205,Areg,Stepanian
55206,Hamzeh,Al Aswad


## Elimino winner_entry e loser_entry perchè non richiesti nel DW

In [9]:
#Elimino winner_entry e loser_entry perchè non richiesti nel DW
tennis=tennis.drop(['winner_entry', 'loser_entry'], axis=1)

## Problema con gli ID dei player


49 player presentano stesso ID ma nome differente. In un caso l'errore è nella spaziatura del nome; in un altro caso l'errore è in una lettera nel nome; negli altri 47 casi sono giocatori diversi con stesso ID

In [10]:
#Creao un nuovo dataframe riferito ai diversi player, con id e nome, concatenando i valori 
#sugli attributi winner_id e loser_id
winner=tennis[['winner_id','winner_name']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name'})    
loser=tennis[['loser_id','loser_name']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name'})
player=[winner,loser]
player = pd.concat(player)
player




,player_id,player_name
0,105453,Kei Nishikori
1,106421,Daniil Medvedev
2,105453,Kei Nishikori
3,104542,Jo-Wilfried Tsonga
4,106421,Daniil Medvedev
...,...,...
186068,222081,Eleana Yu
186069,220723,Jessica Livianu
186070,202478,Tara Moore
186071,205922,Chanelle Van Nguyen


In [11]:
##elimino le righe duplicate con stesso ID e stesso nome
player=player.drop_duplicates(subset=['player_id', 'player_name'])
#faccio una groupby su player_id per vedere quanti nomi diversi sono associati allo stesso id
player_Error_ID=player.groupby(['player_id'])['player_id'].count()
#mantengo solo quelli che hanno più di un nome associato
player_Error_ID=player_Error_ID[player_Error_ID>1]
print(player_Error_ID)
print('')

print('Totale ID con nomi diversi = ',end='')
print(len(player_Error_ID))

player_id
200033    2
202273    2
202277    2
202349    2
202359    2
202414    2
202419    2
202420    2
202421    2
202426    2
202428    2
202429    2
202432    2
202433    2
202434    2
202436    2
202437    2
202440    2
202443    2
202447    2
202475    2
202477    2
205831    2
205842    2
205941    2
205984    2
206000    2
206003    2
206062    2
206091    2
206137    2
206145    2
206171    2
206173    2
206199    2
206258    2
206299    2
206363    2
206377    2
206403    2
206426    2
206430    2
206579    2
207990    2
208147    2
208518    2
208603    2
209866    2
216569    2
Name: player_id, dtype: int64

Totale ID con nomi diversi = 49


In [12]:
##stampo tutte le coppie di nomi con stesso ID
for index, value in player_Error_ID.items():
    
    print(str(index) + ' --> ',end='')

    rows=player.loc[player['player_id']==index]
    first=True
    for index_player, row_player in rows.iterrows():
        if first:
            print(row_player['player_name'] + ' - ',end='')
            first=False
        else:
            print(row_player['player_name'])

200033 --> Serena Williams - Kushaan Nath
202273 --> Liga Dekmeijere - Anurag Nenwani
202277 --> Mariana Correa - Damien Wenger
202349 --> Rushmi Chakravarthi - Kai Lemstra
202359 --> Chin Wei Chan - Marvin Moeller
202414 --> Catherine Harrison - Alexandre Rotsaert
202419 --> Barbara Luz - Justin Schlageter
202420 --> An Sophie Mestach - Aidan Mchugh
202421 --> Alison Van Uytvanck - George Houghton
202426 --> Julia Boserup - Matt Halpin
202428 --> Kiki Bertens - Joao Pedro Alcantara
202429 --> Camila Giorgi - David Lucas Ambrozic
202432 --> Hilda Melander - Nick Stachowiak
202433 --> Anna Giulia Remondina - Cong Tien Le
202434 --> Aleksandra Krunic - Ricardo Ortiz
202436 --> Reka Luca Jani - Zi Yang Cai
202437 --> Anastasia Grymalska - Yang Liu
202440 --> Paula Ormaechea - Marco Miceli
202443 --> Valeria Savinykh - Tobias Sonne
202447 --> Alexandra Mueller - Naru Shirafuji
202475 --> Philip Henning - Jasmina Tinjic
202477 --> Dinah Pfizenmaier - Milan Radojkovic
205831 --> Isabella Rob

In [13]:
#vado a vedere quale è il valore massimo tra tutti gli ID di player, in modo da poter assegnare dei nuovi ID
#ai giocatori presenti in player_Error
max_Id=player['player_id'].max()
    
#itero player_Error_ID che è una Series con index uguale all'ID per andare a selezionare gli ID doppioni
#nel dataframe player
for index, value in player_Error_ID.items():
    
    # a questo specifico ID corrispondono 2 nomi per un errore di spaziatura
    if index==216569:
        colonne=['winner','loser']
        for col in colonne:
            rowsPlayerName=tennis.loc[tennis[col+'_name']=='Zeynep  Sena Sarioglan']
            for index_tennis, row_tennis in rowsPlayerName.iterrows():
                #aggiorno il valore
                tennis.at[index_tennis, col+'_name'] = 'Zeynep Sena Sarioglan'
    
    #a questo ID corrispondono 2 nomi diversi per un errore di una vocale
    if index==206403:
        colonne=['winner','loser']
        for col in colonne:
            rowsPlayerName=tennis.loc[tennis[col+'_name']=='Alona Fomina']
            for index_tennis, row_tennis in rowsPlayerName.iterrows():
                #aggiorno il valore
                tennis.at[index_tennis, col+'_name'] = 'Alena Fomina'
    
    
    #per ID==index seleziono le righe della tabella player con quell'ID in modo da recuperare i nomi
    rows=player.loc[player['player_id']==index]
    #questo first mi serve per non modificare l'ID del primo giocatore. Modifico solo quelli dal secondo in poi
    first=True
    #aumento di uno il massimo dell'ID, cosi da avere un nuovo ID non utilizzato nel dataframe
    max_Id=max_Id+1
    #itero sui diversi nomi presenti nel dataframe player con stesso ID
    for index_player, row_player in rows.iterrows():
        #il primo non lo modifico
        if first:
            first=False
            continue
        else:
            #vado a modificare il valore dell'ID su winner_id e loser_id nella tabella tennis,
            #andando prima ad individuare le righe dove è presente quel nome e poi iterando su di esse
            colonne=['winner','loser']
            for col in colonne:
                rowsPlayerName=tennis.loc[tennis[col+'_name']==row_player['player_name']]
                for index_tennis, row_tennis in rowsPlayerName.iterrows():
                    #aggiorno il valore
                    tennis.at[index_tennis, col+'_id'] = max_Id
                
            

In [14]:
#controllo che non ci siano più ID con nomi diversi
winner=tennis[['winner_id','winner_name']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name'})
          
loser=tennis[['loser_id','loser_name']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name'})

player=[winner,loser]
player = pd.concat(player)

##elimino le righe duplicate con stesso ID e stesso nome
player=player.drop_duplicates(subset=['player_id', 'player_name'])
#faccio una groupby su player_id per vedere quanti nomi diversi sono associati allo stesso id
player_Error_ID=player.groupby(['player_id'])['player_id'].count()
#mantengo solo quelli che hanno più di un nome associato
player_Error_ID=player_Error_ID[player_Error_ID>1]
print(player_Error_ID)
print('')

print('Totale ID con nomi diversi = ',end='')
print(len(player_Error_ID))


Series([], Name: player_id, dtype: int64)

Totale ID con nomi diversi = 0


19 player presentano stesso Nome ma ID differente

In [15]:
##faccio una groupby su player_name per vedere quanti ID diversi sono associati allo stesso nome
player_Error_Name=player.groupby(['player_name'])['player_name'].count()
#mantengo solo quelli che hanno più di un ID associato
player_Error_Name=player_Error_Name[player_Error_Name>1]

print(player_Error_Name)
print('')
print('Totale nomi con ID diversi = ',end='')
print(len(player_Error_Name))


player_name
Astrid Cirotte             2
Ekaterina Makarova         2
Eleni Fasoula              2
Ellie Myers                2
Emilse Lujan Ruiz          2
Fitriani Sabatini          2
Giuseppe Tresca            2
Guy Stokman                2
Hei Ching Claudia Ng       2
Holly Fischer              2
Kuan Yi Lee                2
Lara Onal                  2
Liisa Varul                2
Maria Fernanda Navarro     2
Matylda Burylo             2
Nana Kawagishi             2
Sofia Nahiara Garcia       2
Stanislava Bobrovnikova    2
Valentina Lia              2
Name: player_name, dtype: int64

Totale nomi con ID diversi = 19


In [16]:
##stampo tutte le coppie di ID con stesso nome
for index, value in player_Error_Name.items():
    
    print(str(index) + ' --> ',end='')

    rows=player.loc[player['player_name']==index]
    first=True
    for index_player, row_player in rows.iterrows():
        if first:
            print(str(row_player['player_id']) + ' - ',end='')
            first=False
        else:
            print(row_player['player_id'])
        

Astrid Cirotte --> 236980 - 223401
Ekaterina Makarova --> 201505 - 223126
Eleni Fasoula --> 222943 - 222942
Ellie Myers --> 222407 - 240168
Emilse Lujan Ruiz --> 222646 - 236974
Fitriani Sabatini --> 221189 - 239429
Giuseppe Tresca --> 207982 - 206834
Guy Stokman --> 206883 - 207142
Hei Ching Claudia Ng --> 219835 - 216586
Holly Fischer --> 221142 - 222228
Kuan Yi Lee --> 134120 - 221745
Lara Onal --> 224486 - 223215
Liisa Varul --> 222914 - 222915
Maria Fernanda Navarro --> 223367 - 223287
Matylda Burylo --> 222845 - 222066
Nana Kawagishi --> 221771 - 244078
Sofia Nahiara Garcia --> 220928 - 236977
Stanislava Bobrovnikova --> 221347 - 225865
Valentina Lia --> 220334 - 239415


In [17]:
#itero player_Error_Name che è una Series con index uguale al nome per andare a selezionare i nomi 
#nel dataframe player con differenti ID
for index, value in player_Error_Name.items():

    #per name==index seleziono le righe della tabella player con quel nome in modo da recuperare gli ID
    rows=player.loc[player['player_name']==index]
    #questo first mi serve per non modificare l'ID del primo giocatore. Modifico solo quelli dal secondo in poi
    #mettendoli uguali al primo
    first=True
    id_Player=''
    
    #itero sui diversi ID presenti nel dataframe player con stesso nome
    for index_player, row_player in rows.iterrows():

        #il primo non lo modifico
        if first:
            first=False
            id_Player=row_player['player_id']
            continue
        else:
            #vado a modificare il valore dell'ID su winner_id e loser_id nella tabella tennis,
            #andando prima ad individuare le righe dove è presente l'ID da modificare e poi iterando su di esse
            colonne=['winner','loser']
            for col in colonne:
                rowsPlayerName=tennis.loc[tennis[col+'_id']==row_player['player_id']]
                for index_tennis, row_tennis in rowsPlayerName.iterrows():
                    #aggiorno il valore
                    tennis.at[index_tennis, col+'_id'] = id_Player
                    
                    
                    
                    
    

In [18]:
#controllo che non ci siano più nomi con ID diversi
winner=tennis[['winner_id','winner_name']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name'})
          
loser=tennis[['loser_id','loser_name']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name'})

player=[winner,loser]
player = pd.concat(player)


##elimino le righe duplicate con stesso ID e stesso nome
player=player.drop_duplicates(subset=['player_id', 'player_name'])

##faccio una groupby su player_name per vedere quanti ID diversi sono associati allo stesso nome
player_Error_Name=player.groupby(['player_name'])['player_name'].count()
#mantengo solo quelli che hanno più di un ID associato
player_Error_Name=player_Error_Name[player_Error_Name>1]

print(player_Error_Name)
print('')
print('Totale nomi con ID diversi = ',end='')
print(len(player_Error_Name))



Series([], Name: player_name, dtype: int64)

Totale nomi con ID diversi = 0


sono presenti 6 righe con una partita tra giocatori con lo stesso nome. Elimino queste 6 righe

In [19]:
##leggo tutto il dataframe per riga, se winner_name e loser_ name sono uguali aggiungo 
#l'index della riga alla lista
lista=[]
for index, row in tennis.iterrows():
    if row['winner_name']==row['loser_name']:
        print('index riga '+str(index)+' --> partita tra '+row['winner_name']+ ' e '+row['loser_name'])
        lista.append(index)
tennis.loc[lista]

index riga 72793 --> partita tra Steven De Waard e Steven De Waard
index riga 75235 --> partita tra Takuto Niki e Takuto Niki
index riga 75636 --> partita tra Andrei Vasilevski e Andrei Vasilevski
index riga 133100 --> partita tra Lyubov Kostenko e Lyubov Kostenko
index riga 135577 --> partita tra Sophia Fry e Sophia Fry
index riga 179935 --> partita tra Liisa Varul e Liisa Varul


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
72793,2016-7182,Happy Valley CH,Hard,32,C,20160104,244,122059,Steven De Waard,R,NaN,AUS,24.668036,122059,Steven De Waard,R,NaN,AUS,24.668036,6-7(3) 6-4 6-2,3,Q1,64.0,7.0,5.0,82.0,44.0,33.0,24.0,15.0,1.0,3.0,7.0,5.0,82.0,44.0,33.0,24.0,15.0,1.0,3.0,1161.0,8.0,1161.0,8.0,2730,554426.23
75235,2016-7553,Chengdu CH,Hard,32,C,20160801,261,105026,Takuto Niki,U,NaN,JPN,28.804928,105026,Takuto Niki,U,NaN,JPN,28.804928,6-0 6-0,3,Q1,55.0,2.0,1.0,33.0,23.0,20.0,6.0,6.0,2.0,2.0,2.0,1.0,33.0,23.0,20.0,6.0,6.0,2.0,2.0,509.0,71.0,509.0,71.0,3551,551056.44
75636,2016-7614,Astana CH,Hard,32,C,20161121,249,105781,Andrei Vasilevski,R,NaN,BLR,25.486653,105781,Andrei Vasilevski,R,NaN,BLR,25.486653,6-1 6-3,3,Q1,55.0,5.0,1.0,41.0,22.0,19.0,10.0,8.0,0.0,1.0,5.0,1.0,41.0,22.0,19.0,10.0,8.0,0.0,1.0,741.0,28.0,741.0,28.0,3135,555708.14
133100,2018-W-WITF-UKR-02A-2018,Chornomorsk $15K,Clay,32,15,20181001,6,221371,Lyubov Kostenko,R,NaN,UKR,15.512663,221371,Lyubov Kostenko,R,NaN,UKR,15.512663,6-2 6-4,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3130,564093.49
135577,2019-W-ITF-AUS-04A-2019,W15 Port Pirie,Hard,32,15,20190211,15,221862,Sophia Fry,R,NaN,AUS,18.532512,221862,Sophia Fry,R,NaN,AUS,18.532512,6-0 6-2,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3243,558383.46
179935,2020-W-ITF-EST-01A-2020,W15 Haabneeme,Hard,32,15,20201116,110,222914,Liisa Varul,U,NaN,EST,NaN,222914,Liisa Varul,U,NaN,EST,NaN,W/O,3,Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3475,705886.05


In [20]:
#elimino questi 6 record
tennis.drop(lista, inplace=True)

## inserisco in male e female i nomi mancanti presenti in tennis

In [21]:
#prima di tutto osservo che ci sono alcuni nomi di player sia in female che in male
inter = male_set.intersection(female_set)
print(len(inter))
print (inter)

64
{'M Bonner', 'M Williams', 'Ziyu Zhang', 'P Dyson', 'P Smith', 'J Neville', 'J Moran', 'S Sullivan', 'H Brown', 'I Baker', 'Yang Liu', 'E Barker', 'Hui Zhang', 'M Ferenczy', 'A Smith', 'Bunny Austin', 'R Lyons', 'J Lambert', 'M Noble', 'J Robinson', 'P Scott', 'Robin White', 'J Tobin', 'R Weber', 'J Walker', 'C Ward', 'J Nicholson', 'L Darby', 'Hao Zhang', 'J May', 'R Mortoire', 'L Robinson', 'B Butcher', 'M Bruce', 'Di Zhao', 'D Hall', 'S Davis', 'J Thomson', 'Yi Liu', 'J Smith', 'A Jackson', 'A Hall', 'Robin Schneider', 'D Turner', 'Peter Smith', 'N Willman', 'C Mann', 'Austin Smith', 'P Moore', 'G Roberts', 'B Johnson', 'A Foster', 'J Lacey', 'Kuan Yi Lee', 'J Young', 'M Thomas', 'B Morgan', 'M Graham', 'Yi Yang', 'Xin Yang', 'K Roberts', 'K Williams', 'M Hamilton', 'Bobby Riggs'}


In [22]:
male

,name,surname
0,Gardnar,Mulloy
1,Pancho,Segura
2,Frank,Sedgman
3,Giuseppe,Merlo
4,Richard Pancho,Gonzales
...,...,...
55203,Borys,Zgola
55204,Noah,Schlagenhauf
55205,Areg,Stepanian
55206,Hamzeh,Al Aswad


In [23]:
#per eliminare i nomi su entrambe le tabelle, creo altre due copie di tabelle con la concatenzione
#di nome e cognome. Questo mi serve per risalire all'index partendo da una stringa dove ho concatenato
#nome e cognome
male_2=male.copy()
male_2['nome']=male['name']+' '+male['surname']
male_2

,name,surname,nome
0,Gardnar,Mulloy,Gardnar Mulloy
1,Pancho,Segura,Pancho Segura
2,Frank,Sedgman,Frank Sedgman
3,Giuseppe,Merlo,Giuseppe Merlo
4,Richard Pancho,Gonzales,Richard Pancho Gonzales
...,...,...,...
55203,Borys,Zgola,Borys Zgola
55204,Noah,Schlagenhauf,Noah Schlagenhauf
55205,Areg,Stepanian,Areg Stepanian
55206,Hamzeh,Al Aswad,Hamzeh Al Aswad


In [24]:
female_2=female.copy()
female_2['nome']=female['name']+' '+female['surname']
female_2

,name,surname,nome
0,Bobby,Riggs,Bobby Riggs
1,X,X,X X
2,Martina,Hingis,Martina Hingis
3,Mirjana,Lucic,Mirjana Lucic
4,Justine,Henin,Justine Henin
...,...,...,...
46167,Kathleen,Allen,Kathleen Allen
46168,Mrs,Dransfield,Mrs Dransfield
46169,Jessie,Greene,Jessie Greene
46170,Mary,Trumbull,Mary Trumbull


In [25]:
#elimino questi valori da entrambi i set. Andrò poi a calcolare in base alla partita
#dove aggiungerli. Li elimino anche dai file male e female attraverso l'index
for name in inter:
    female_set.remove(name)
    rows=female_2.loc[female_2['nome'] == name]
    for index, row in rows.iterrows():
        female=female.drop([index])
    male_set.remove(name)
    rows=male_2.loc[male_2['nome'] == name]
    for index, row in rows.iterrows():
       male=male.drop([index])

In [26]:
#vado a vedere quali player di tennis non sono presenti nei due set 

#creo un dataframe con tutti i player presenti in tennis con id e name. Prendo i dati dai 
#campi di winner e loser su ogni riga
winner=tennis[['winner_id','winner_name']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name'})
          
loser=tennis[['loser_id','loser_name']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name'})

player=[winner,loser]
player = pd.concat(player)
#elimino i duplicati, cioè player che hanno giocato più di una partita
player=player.drop_duplicates(subset=['player_id', 'player_name'])

#lista dove andrò ad inserire i nomi dei giocatori presenti in tennis ma non in male o female
lista=list()

#se il player non è presente ne in male_set ne in female_set, allora stampo il nome e lo inserisco in lista
a=0
for index, row in player.iterrows():
    if row['player_name'] in male_set:
        continue
    elif row['player_name'] in female_set:
        continue
    else:
        lista.append(row['player_name'])
        a=a+1
        print(row['player_name']+' non ha valore su sex')
print(a)

Jo-Wilfried Tsonga non ha valore su sex
Cristian Garin non ha valore su sex
Stan Wawrinka non ha valore su sex
Antoine Hoang non ha valore su sex
Taylor Fritz non ha valore su sex
Mackenzie McDonald non ha valore su sex
Diego Schwartzman non ha valore su sex
Frances Tiafoe non ha valore su sex
Pedro Martinez non ha valore su sex
Lloyd Harris non ha valore su sex
Franko Skugor non ha valore su sex
Juan Martin del Potro non ha valore su sex
J.J. Wolf non ha valore su sex
Daniel Elahi Galan non ha valore su sex
Joao Menezes non ha valore su sex
Jurabek Karimov non ha valore su sex
Juan Pablo Varillas non ha valore su sex
Holger Rune non ha valore su sex
Khumoun Sultanov non ha valore su sex
Evgenii Tiurnev non ha valore su sex
Di Zhao non ha valore su sex
Christopher O'Connell non ha valore su sex
Sam Groth non ha valore su sex
Daniel Munoz de la Nava non ha valore su sex
Botic van de Zandschulp non ha valore su sex
Austin Smith non ha valore su sex
Alexandar Lazov non ha valore su sex
Da

In [27]:
male

,name,surname
0,Gardnar,Mulloy
1,Pancho,Segura
2,Frank,Sedgman
3,Giuseppe,Merlo
4,Richard Pancho,Gonzales
...,...,...
55203,Borys,Zgola
55204,Noah,Schlagenhauf
55205,Areg,Stepanian
55206,Hamzeh,Al Aswad


In [28]:
female

,name,surname
1,X,X
2,Martina,Hingis
3,Mirjana,Lucic
4,Justine,Henin
5,Kerry Anne,Guse
...,...,...
46167,Kathleen,Allen
46168,Mrs,Dransfield
46169,Jessie,Greene
46170,Mary,Trumbull


In [29]:

# funzione che tokenizza il nome in modo da dividerlo in name e surname
def split_players(name):
    nome_token=name.split()
    if len(nome_token)==2:
        new_row={'name':nome_token[0],'surname':nome_token[1]}
    elif len(nome_token)==3:
        new_row={'name':nome_token[0]+' '+nome_token[1],'surname':nome_token[2]}
    elif len(nome_token)==4:
        new_row={'name':nome_token[0]+' '+nome_token[1],'surname':nome_token[2]+' '+nome_token[3]}
    elif len(nome_token)==5:
        new_row={'name':nome_token[0]+' '+nome_token[1],'surname':nome_token[2]+' '+nome_token[3]+' '+nome_token[4]}
    else:
        print('ci sono più di 5 token per il player '+player)
        return ' '
    return new_row


In [30]:
# vado a scorrere il dataframe tennis; se un giocare è in lista (cioè non è presente ne in male ne in female)
# controllo l'altro giocatore con cui sta giocando. In base all'altro capisco il sesso. Aggiungo
#il nome al set corrispondente ed aggiungo la row al dataframe male o female
lista_non_aggiunti=[]
for index, row in tennis.iterrows():
    if len(lista)==0:
        break
    if row['winner_name'] in lista:
        if row['loser_name'] in male_set:
            male_set.add(row['winner_name'])
            new_row=split_players(row['winner_name'])
            male=male.append(new_row, ignore_index=True)
            lista.remove(row['winner_name'])

        elif row['loser_name'] in female_set:
            female_set.add(row['winner_name'])
            new_row=split_players(row['winner_name'])
            female=female.append(new_row, ignore_index=True)
            lista.remove(row['winner_name'])
        else:
            lista_non_aggiunti.append(row['winner_name'])
            print(row['winner_name'])
        
        
    if row['loser_name'] in lista:
        if row['winner_name'] in male_set:
            male_set.add(row['loser_name'])
            new_row=split_players(row['loser_name'])
            male=male.append(new_row, ignore_index=True)
            lista.remove(row['loser_name'])
        elif row['winner_name'] in female_set:
            female_set.add(row['loser_name'])
            new_row=split_players(row['loser_name'])
            female=female.append(new_row, ignore_index=True)
            lista.remove(row['loser_name'])

        else:
            lista_non_aggiunti.append(row['loser_name'])
            print(row['loser_name'])        
    else:
        continue

Cristian Garin
Christopher O'Connell


In [31]:
lista_non_aggiunti

['Cristian Garin', "Christopher O'Connell"]

In [32]:
for i in lista_non_aggiunti:
    if i in male_set or i in female_set:
        continue
    else:
        print (i)

In [33]:
male

,name,surname
0,Gardnar,Mulloy
1,Pancho,Segura
2,Frank,Sedgman
3,Giuseppe,Merlo
4,Richard Pancho,Gonzales
...,...,...
54993,Ben,Patael
54994,Austin,Smith
54995,David,O'Hare
54996,Yang,Liu


In [34]:
female


,name,surname
0,X,X
1,Martina,Hingis
2,Mirjana,Lucic
3,Justine,Henin
4,Kerry Anne,Guse
...,...,...
44432,Mary,Trumbull
44433,Inga,Sperling
44434,Di,Zhao
44435,Yi,Liu


## PROBLEMA CON HAND DEI PLAYER

In [35]:
tennis['winner_hand'].value_counts()

R    120369
U     49114
L     16259
Name: winner_hand, dtype: int64

In [36]:
tennis['loser_hand'].value_counts()

R    109059
U     61866
L     14762
Name: loser_hand, dtype: int64

In [37]:
#setto per i null value il valore U che corrisponde a Undefined
tennis['winner_hand'] = tennis['winner_hand'].fillna('U')
tennis['loser_hand'] = tennis['loser_hand'].fillna('U')

In [38]:

winner=tennis[['winner_id','winner_name','winner_hand']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_hand':'player_hand'})
          
loser=tennis[['loser_id','loser_name','loser_hand']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_hand':'player_hand'})

player=[winner,loser]
player = pd.concat(player)
player

,player_id,player_name,player_hand
0,105453,Kei Nishikori,R
1,106421,Daniil Medvedev,R
2,105453,Kei Nishikori,R
3,104542,Jo-Wilfried Tsonga,R
4,106421,Daniil Medvedev,R
...,...,...,...
186068,222081,Eleana Yu,R
186069,220723,Jessica Livianu,R
186070,202478,Tara Moore,R
186071,205922,Chanelle Van Nguyen,U


In [39]:
player=player.drop_duplicates(subset=['player_id', 'player_name','player_hand'])
player_Error_Hand=player.groupby(['player_name'])['player_name'].count()
player_Error_Hand=player_Error_Hand[player_Error_Hand>1]

print(player_Error_Hand)

print('')
print('Totale nomi con hand diverse = ',end='')
print(len(player_Error_Hand))



player_name
Amina Anshba            2
Anastasia Kulikova      2
Ekaterina Makarova      2
Eva Shaw                2
Hei Ching Claudia Ng    2
Qinwen Zheng            2
Vanessa Ersoz           2
Weronika Baszak         2
Name: player_name, dtype: int64

Totale nomi con hand diverse = 8


In [40]:
##stampo tutti i player con diverso valori su hand
for index, value in player_Error_Hand.items():
    
    print(str(index) + ' --> ',end='')

    rows=player.loc[player['player_name']==index]
    first=True
    for index_player, row_player in rows.iterrows():
        if first:
            print(str(row_player['player_hand']) + ' - ',end='')
            first=False
        else:
            print(row_player['player_hand'])

Amina Anshba --> U - L
Anastasia Kulikova --> U - L
Ekaterina Makarova --> L - U
Eva Shaw --> U - R
Hei Ching Claudia Ng --> U - R
Qinwen Zheng --> R - U
Vanessa Ersoz --> R - U
Weronika Baszak --> R - U


In [41]:
#PER QUESTI 8 GIOCATORI VADO A SOSTITUIRE IL VALORE U (undefined) CON R OPPURE L IN BASE AI VALORI PRESENTI
for index, value in player_Error_Hand.items():
    
    #per name==index seleziono le righe della tabella player con quel nome in modo da recuperare il valore R o L per hand
    rows=player.loc[player['player_name']==index]
    hand='U'
    for index_player, row_player in rows.iterrows():
       #cerco un valore diverso da U, appena lo trovo esco dal ciclo
        if row_player['player_hand'] != 'U':
            hand=row_player['player_hand']
            break 
           
    #se non trova valori diversi da U termina per quel player (non dovrebbe accadere in questo caso)
    if hand== 'U':
            continue
    else:
       
        #vado a modificare il valore di hand su winner_hand e loser_hand nella tabella tennis,
        #andando prima ad individuare le righe dove è presente il nome del player da modificare e poi iterando su di esse
        colonne=['winner','loser']
        for col in colonne:
            rowsPlayerName=tennis.loc[tennis[col+'_name']==index]
            for index_tennis, row_tennis in rowsPlayerName.iterrows():
                #aggiorno il valore
                tennis.at[index_tennis, col+'_hand'] = hand


In [42]:
#controllo che non ci siano giocatori con tesso nome e diversa hand
winner=tennis[['winner_id','winner_name','winner_hand']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_hand':'player_hand'})
          
loser=tennis[['loser_id','loser_name','loser_hand']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_hand':'player_hand'})

player=[winner,loser]
player = pd.concat(player)

player=player.drop_duplicates(subset=['player_id', 'player_name','player_hand'])
player_Error_Hand=player.groupby(['player_name'])['player_name'].count()
player_Error_Hand=player_Error_Hand[player_Error_Hand>1]

print(player_Error_Hand)

print('')
print('Totale nomi con ID diversi = ',end='')
print(len(player_Error_Hand))

Series([], Name: player_name, dtype: int64)

Totale nomi con ID diversi = 0


## PROBLEMA CON IOC DI PLAYER

sono presenti 4 player con diverso valore su IOC

In [43]:
winner=tennis[['winner_id','winner_name','winner_ioc']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_ioc':'player_ioc'})
          
loser=tennis[['loser_id','loser_name','loser_ioc']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_ioc':'player_ioc'})

player=[winner,loser]
player = pd.concat(player)
player=player.drop_duplicates(subset=['player_id', 'player_name','player_ioc'])
player

,player_id,player_name,player_ioc
0,105453,Kei Nishikori,JPN
1,106421,Daniil Medvedev,RUS
3,104542,Jo-Wilfried Tsonga,FRA
5,104871,Jeremy Chardy,FRA
8,200282,Alex De Minaur,AUS
...,...,...,...
185021,223315,Dilara Sultanova,TKM
185560,223075,Madeleine Jessup,USA
185780,223216,Katie Codd,USA
185781,223217,Amy Huang,USA


In [44]:
player_Error_IOC=player.groupby(['player_name'])['player_name'].count()
player_Error_IOC=player_Error_IOC[player_Error_IOC>1]

print(player_Error_IOC)

print('')
print('Totale nomi con IOC diversi = ',end='')
print(len(player_Error_IOC))


player_name
Alena Fomina          2
Anastasia Kulikova    2
Eleni Fasoula         2
Matylda Burylo        2
Name: player_name, dtype: int64

Totale nomi con IOC diversi = 4


In [45]:
##stampo tutti i player con diverso valori su ioc
for index, value in player_Error_IOC.items():
    
    print(str(index) + ' --> ',end='')

    rows=player.loc[player['player_name']==index]
    first=True
    for index_player, row_player in rows.iterrows():
        if first:
            print(str(row_player['player_ioc']) + ' - ',end='')
            first=False
        else:
            print(row_player['player_ioc'])

Alena Fomina --> UKR - RUS
Anastasia Kulikova --> RUS - FIN
Eleni Fasoula --> KAZ - GRE
Matylda Burylo --> POL - NOR


In [46]:
#non sapendo il valore da scegliere ed essendo solo 4 casi, li abbiamo scelti guardando su google
dizionario={'Alena Fomina':'RUS','Anastasia Kulikova':'FIN','Eleni Fasoula':'GRE','Matylda Burylo':'NOR'}
colonne=['winner','loser']
#itero sul dizionario e vado a modificare i valori di oic su winner e loser
for index in dizionario:
    for col in colonne:
            rowsPlayerName=tennis.loc[tennis[col+'_name']==index]
            for index_tennis, row_tennis in rowsPlayerName.iterrows():
                #aggiorno il valore
                tennis.at[index_tennis, col+'_ioc'] = dizionario[index]
                
                


In [47]:
#controllo se ho risolto i problemi su IOC
winner=tennis[['winner_id','winner_name','winner_ioc']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_ioc':'player_ioc'})
          
loser=tennis[['loser_id','loser_name','loser_ioc']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_ioc':'player_ioc'})

player=[winner,loser]
player = pd.concat(player)
player=player.drop_duplicates(subset=['player_id', 'player_name','player_ioc'])


player_Error_IOC=player.groupby(['player_name'])['player_name'].count()
player_Error_IOC=player_Error_IOC[player_Error_IOC>1]

player_Error_IOC.sort_values()

Series([], Name: player_name, dtype: int64)

## PROBLEMA CON HT DEI PLAYER

In [48]:
winner=tennis[['winner_id','winner_name','winner_ht']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_ht':'player_ht'})
          
loser=tennis[['loser_id','loser_name','loser_ht']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_ht':'player_ht'})

player=[winner,loser]
player = pd.concat(player)
player=player.drop_duplicates(subset=['player_id', 'player_name','player_ht'])
player

,player_id,player_name,player_ht
0,105453,Kei Nishikori,178.0
1,106421,Daniil Medvedev,198.0
3,104542,Jo-Wilfried Tsonga,188.0
5,104871,Jeremy Chardy,188.0
8,200282,Alex De Minaur,183.0
...,...,...,...
185021,223315,Dilara Sultanova,NaN
185560,223075,Madeleine Jessup,NaN
185780,223216,Katie Codd,NaN
185781,223217,Amy Huang,NaN


In [49]:

player_Error_HT=player.groupby(['player_name'])['player_name'].count()
player_Error_HT=player_Error_HT[player_Error_HT>1]


print(player_Error_HT)

print('')
print('Totale nomi con ht diversi = ',end='')
print(len(player_Error_HT))

player_name
Ajla Tomljanovic          2
Amanda Anisimova          2
Anastasia Potapova        2
Anastasija Sevastova      2
Ann Li                    2
Danka Kovinic             2
David Goffin              2
Donna Vekic               2
Ekaterina Makarova        2
Emma Raducanu             2
Jil Belen Teichmann       2
Kaja Juvan                2
Kamilla Rakhimova         2
Laura Siegemund           2
Leylah Annie Fernandez    2
Ludmilla Samsonova        2
Marta Kostyuk             2
Martina Trevisan          2
Rebecca Peterson          2
Sara Sorribes Tormo       2
Viktorija Golubic         2
Name: player_name, dtype: int64

Totale nomi con ht diversi = 21


In [50]:
##stampo tutti i player con diverso valori su ht
for index, value in player_Error_HT.items():
    
    print(str(index) + ' --> ',end='')

    rows=player.loc[player['player_name']==index]
    first=True
    for index_player, row_player in rows.iterrows():
        if first:
            print(str(row_player['player_ht']) + ' - ',end='')
            first=False
        else:
            print(row_player['player_ht'])

Ajla Tomljanovic --> nan - 180.0
Amanda Anisimova --> nan - 180.0
Anastasia Potapova --> nan - 175.0
Anastasija Sevastova --> nan - 169.0
Ann Li --> nan - 170.0
Danka Kovinic --> nan - 169.0
David Goffin --> 180.0 - 163.0
Donna Vekic --> nan - 179.0
Ekaterina Makarova --> 178.0 - nan
Emma Raducanu --> 175.0 - nan
Jil Belen Teichmann --> nan - 170.0
Kaja Juvan --> nan - 170.0
Kamilla Rakhimova --> 2.0 - nan
Laura Siegemund --> nan - 168.0
Leylah Annie Fernandez --> nan - 168.0
Ludmilla Samsonova --> nan - 180.0
Marta Kostyuk --> nan - 175.0
Martina Trevisan --> nan - 160.0
Rebecca Peterson --> nan - 173.0
Sara Sorribes Tormo --> nan - 176.0
Viktorija Golubic --> nan - 169.0


David Goffin presenta diversi valori sull'altezza. Gli altri 20 presentano valori diversi solo perchè presentano valori null. Kamilla Rakhimova presenta inoltre un valore di 2 cm evidentemente errato

In [51]:
#PER QUESTI GIOCATORI VADO A SOSTITUIRE IL VALORE NULL L'ALTRO VALORE PRESENTE. 
#IL VALORE CORRETTO PER DAVID GOFFIN E' STATO RICERCATO SU GOOGLE
for index, value in player_Error_HT.items():
    
    
    # a questo specifico name corrispondono due valori, quello corretto è 180
    if index=='David Goffin':
        colonne=['winner','loser']
        for col in colonne:
            rowsPlayerName=tennis.loc[tennis[col+'_name']=='David Goffin']
            for index_tennis, row_tennis in rowsPlayerName.iterrows():
                #aggiorno il valore
                tennis.at[index_tennis, col+'_ht'] = 180
    
    # a questo specifico name corrispondono due valori, uno null e uno pari a 2 cm. Essendo errato il valore 2 cm, setto 
    #tutto a null
    if index=='Kamilla Rakhimova':
        colonne=['winner','loser']
        for col in colonne:
            rowsPlayerName=tennis.loc[tennis[col+'_name']=='Kamilla Rakhimova']
            for index_tennis, row_tennis in rowsPlayerName.iterrows():
                #aggiorno il valore
                tennis.at[index_tennis, col+'_ht'] = np.nan
    
    
    
    #per name==index seleziono le righe della tabella player con quel nome in modo da recuperare il valore dell'altezza
    rows=player.loc[player['player_name']==index]
    ht=0
    for index_player, row_player in rows.iterrows():
       #cerco un valore non null, appena lo trovo esco dal ciclo
        if pd.isna(row_player['player_ht']):
            continue
        else:
            ht=row_player['player_ht']
            break 
    #se non trova valori diversi da null termina per quel player (non dovrebbe accadere in questo caso)
    if ht== 0:
            continue
    else:
       
        #vado a modificare il valore di ht su winner_ht e loser_ht nella tabella tennis,
        #andando prima ad individuare le righe dove è presente il nome del player da modificare e poi iterando su di esse
        colonne=['winner','loser']
        for col in colonne:
            rowsPlayerName=tennis.loc[tennis[col+'_name']==index]
            for index_tennis, row_tennis in rowsPlayerName.iterrows():
                #aggiorno il valore
                tennis.at[index_tennis, col+'_ht'] = ht


In [52]:

winner=tennis[['winner_id','winner_name','winner_ht']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_ht':'player_ht'})
          
loser=tennis[['loser_id','loser_name','loser_ht']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_ht':'player_ht'})

player=[winner,loser]
player = pd.concat(player)
player=player.drop_duplicates(subset=['player_id', 'player_name','player_ht'])

player_Error_HT=player.groupby(['player_name'])['player_name'].count()
player_Error_HT=player_Error_HT[player_Error_HT>1]


print(player_Error_HT)

print('')
print('Totale nomi con ht diversi = ',end='')
print(len(player_Error_HT))

Series([], Name: player_name, dtype: int64)

Totale nomi con ht diversi = 0


per riempire i valore null faccio una media dei valori in base al sesso.


In [53]:
male.head()

,name,surname
0,Gardnar,Mulloy
1,Pancho,Segura
2,Frank,Sedgman
3,Giuseppe,Merlo
4,Richard Pancho,Gonzales


In [54]:
#creo una tabella player senza null value, in modo da calcolare le medie per il sesso male e female
winner=tennis[['winner_id','winner_name','winner_ht']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_ht':'player_ht'})
          
loser=tennis[['loser_id','loser_name','loser_ht']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_ht':'player_ht'})

player=[winner,loser]
player = pd.concat(player)
player=player.drop_duplicates(subset=['player_id', 'player_name','player_ht'])
player=player.dropna()
player

,player_id,player_name,player_ht
0,105453,Kei Nishikori,178.0
1,106421,Daniil Medvedev,198.0
3,104542,Jo-Wilfried Tsonga,188.0
5,104871,Jeremy Chardy,188.0
8,200282,Alex De Minaur,183.0
...,...,...,...
95755,201364,Tatiana Golovin,175.0
112247,104583,Filip Polasek,196.0
116251,103657,Ivo Klec,183.0
142416,201431,Aravane Rezai,165.0


In [55]:
#aggiungo a player una colonna con il sesso, ricercando il nome nei due insiemi. Se un nome non è presente nei due insiemi
#setto il sesso a null
player['player_sex']=''
for index, row in player.iterrows():
    if (row['player_name'] in male_set):
        player.at[index, 'player_sex'] = 'male'
    elif (row['player_name']  in female_set):
        player.at[index, 'player_sex'] = 'female'
    else:
        player.at[index, 'player_sex'] = np.nan
        print(str(index)+' non ha il sesso')



In [56]:
#elimino i player con sex null, inutili per calcolare le medie
player=player.dropna()
player

,player_id,player_name,player_ht,player_sex
0,105453,Kei Nishikori,178.0,male
1,106421,Daniil Medvedev,198.0,male
3,104542,Jo-Wilfried Tsonga,188.0,male
5,104871,Jeremy Chardy,188.0,male
8,200282,Alex De Minaur,183.0,male
...,...,...,...,...
95755,201364,Tatiana Golovin,175.0,female
112247,104583,Filip Polasek,196.0,male
116251,103657,Ivo Klec,183.0,male
142416,201431,Aravane Rezai,165.0,female


In [57]:
#calcolo le medie
media_Altezza=player.groupby(['player_sex'])['player_ht'].mean()
media_Altezza

player_sex
female    172.815000
male      185.844575
Name: player_ht, dtype: float64

In [58]:
#tolgo i decimali ma lascio il tipo float
media_donna=float(int(media_Altezza[0]))
media_uomo=float(int(media_Altezza[1]))
#calcolo anche la media per i player senza sesso
media_senzaSesso=float(int(mean([media_donna,media_uomo])))


In [59]:
#itero su tennis, riempiendo i null value con le medie in base al sesso. Ricerco il sesso sui due set
#avendo completato la tabella male con i nomi mancanti nessun player è senza sesso
for index, row in tennis.iterrows():
    
    colonne=['winner','loser']
    for col in colonne:
        if pd.isna(row[col+'_ht']):
            if row[col+'_name'] in male_set:
                altezza=media_uomo
            elif row[col+'_name'] in female_set:
                altezza=media_donna
            else:
                altezza=media_senzaSesso

            tennis.at[index, col+'_ht'] = altezza
       




In [60]:
tennis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185758 entries, 0 to 186072
Data columns (total 47 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          185758 non-null  object 
 1   tourney_name        185758 non-null  object 
 2   surface             185596 non-null  object 
 3   draw_size           185758 non-null  int64  
 4   tourney_level       185758 non-null  object 
 5   tourney_date        185758 non-null  int64  
 6   match_num           185758 non-null  int64  
 7   winner_id           185758 non-null  int64  
 8   winner_name         185758 non-null  object 
 9   winner_hand         185758 non-null  object 
 10  winner_ht           185758 non-null  float64
 11  winner_ioc          185758 non-null  object 
 12  winner_age          182931 non-null  float64
 13  loser_id            185758 non-null  int64  
 14  loser_name          185758 non-null  object 
 15  loser_hand          185758 non-nul

## PROBLEMA AGE

VANNO RIEMPITI I NULL VALUE. Alcuni player presentano il valore di age su alcuni record  ma non su altri. Mentre altri presentano val null su tutti i record. In questo ultimo caso i valori null sono riempiti con la media attualizzata in base al sesso. cioè prima ho calcolato l'età nell'anno 2021 di ogni giocatore, poi ho calcolato la media e successivamente ho calcolato age in base alla data del torneo

In [61]:
#unisco i player con i loro valori su age tra quelli in winner e quelli in loser
winner=tennis[['winner_id','winner_name','winner_age','tourney_date']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_age':'player_age'})
          
loser=tennis[['loser_id','loser_name','loser_age','tourney_date']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_age':'player_age'})

player=[winner,loser]
player = pd.concat(player)

player

,player_id,player_name,player_age,tourney_date
0,105453,Kei Nishikori,29.004791,20181231
1,106421,Daniil Medvedev,22.885695,20181231
2,105453,Kei Nishikori,29.004791,20181231
3,104542,Jo-Wilfried Tsonga,33.705681,20181231
4,106421,Daniil Medvedev,22.885695,20181231
...,...,...,...,...
186068,222081,Eleana Yu,16.002738,20201109
186069,220723,Jessica Livianu,22.713210,20201109
186070,202478,Tara Moore,28.260096,20201109
186071,205922,Chanelle Van Nguyen,26.806297,20201109


In [62]:
#creo una tabella con solo i valori non null
senza_NAN=player.dropna()

senza_NAN

,player_id,player_name,player_age,tourney_date
0,105453,Kei Nishikori,29.004791,20181231
1,106421,Daniil Medvedev,22.885695,20181231
2,105453,Kei Nishikori,29.004791,20181231
3,104542,Jo-Wilfried Tsonga,33.705681,20181231
4,106421,Daniil Medvedev,22.885695,20181231
...,...,...,...,...
186067,211765,Emina Bektas,27.613963,20201109
186068,222081,Eleana Yu,16.002738,20201109
186069,220723,Jessica Livianu,22.713210,20201109
186070,202478,Tara Moore,28.260096,20201109


In [63]:
#creo una tabella con solo valori null

solo_NAN=player[player['player_age'].isna()]
#faccio reset dell'index, altrimenti, perchè avendo concatenato 2 dataframe, ho indici doppi
solo_NAN=solo_NAN.reset_index()
solo_NAN

,index,player_id,player_name,player_age,tourney_date
0,5890,209188,Ari Fahresi,NaN,20190914
1,19065,220019,Venia Yeung,NaN,20170724
2,21732,220257,Nour Dib,NaN,20170925
3,36444,220246,Daria Lukyanova,NaN,20161212
4,37538,220247,Shufaa Changawa Ruwa,NaN,20161212
...,...,...,...,...,...
9322,185786,222947,Maya Pitts,NaN,20200224
9323,185940,223451,Chloe Henderson,NaN,20201026
9324,185999,223452,Emma Charney,NaN,20201102
9325,186009,223318,Whitley Pate,NaN,20201102


In [64]:
#creo un set a partire dalla tabella senza null value. Il set contiene i nomi dei giocatori. 
#In questo modo vado a capire quali sono i player che non presentano mai un valore su age
insieme_Player_Age=set(senza_NAN['player_name'])
a=0
#itero su una lista cosi posso eliminare immediatamente i player che che
#si trovano nel set con valori su età
for index, row in list(solo_NAN.iterrows()):
    if row['player_name'] in insieme_Player_Age:
        a=a+1
        solo_NAN=solo_NAN.drop([index])
        print(row['player_name']+' ha l eta su altro record')
print(a)


Sofia Nahiara Garcia ha l eta su altro record
Valentina Lia ha l eta su altro record
Stanislava Bobrovnikova ha l eta su altro record
Fitriani Sabatini ha l eta su altro record
Fitriani Sabatini ha l eta su altro record
Fitriani Sabatini ha l eta su altro record
Fitriani Sabatini ha l eta su altro record
Fitriani Sabatini ha l eta su altro record
Fitriani Sabatini ha l eta su altro record
Nana Kawagishi ha l eta su altro record
Nana Kawagishi ha l eta su altro record
Nana Kawagishi ha l eta su altro record
Nana Kawagishi ha l eta su altro record
Nana Kawagishi ha l eta su altro record
Nana Kawagishi ha l eta su altro record
Stanislava Bobrovnikova ha l eta su altro record
Eva Shaw ha l eta su altro record
Vanessa Ersoz ha l eta su altro record
Stanislava Bobrovnikova ha l eta su altro record
Hei Ching Claudia Ng ha l eta su altro record
Ekaterina Makarova ha l eta su altro record
Giuseppe Tresca ha l eta su altro record
Giuseppe Tresca ha l eta su altro record
Giuseppe Tresca ha l eta 

In [65]:
rowsPlayerID=tennis.loc[tennis['loser_name']=='Sofia Nahiara Garcia']
rowsPlayerID['loser_age']


135277    16.446270
135305    16.465435
135364    16.522930
163295          NaN
Name: loser_age, dtype: float64

In [66]:
#questi 9265  non hanno mai l'età su nessun record (alcuni si ripetono)
solo_NAN

,index,player_id,player_name,player_age,tourney_date
0,5890,209188,Ari Fahresi,NaN,20190914
1,19065,220019,Venia Yeung,NaN,20170724
2,21732,220257,Nour Dib,NaN,20170925
3,36444,220246,Daria Lukyanova,NaN,20161212
4,37538,220247,Shufaa Changawa Ruwa,NaN,20161212
...,...,...,...,...,...
9322,185786,222947,Maya Pitts,NaN,20200224
9323,185940,223451,Chloe Henderson,NaN,20201026
9324,185999,223452,Emma Charney,NaN,20201102
9325,186009,223318,Whitley Pate,NaN,20201102


In [67]:
rowsPlayerID=tennis.loc[tennis['winner_name']=='Ari Fahresi']
rowsPlayerID['winner_age']


5890   NaN
Name: winner_age, dtype: float64

per riempire i valore null faccio una media dei valori in base al sesso.


In [68]:
senza_NAN=senza_NAN.drop_duplicates(subset=['player_id', 'player_name'])
senza_NAN


,player_id,player_name,player_age,tourney_date
0,105453,Kei Nishikori,29.004791,20181231
1,106421,Daniil Medvedev,22.885695,20181231
3,104542,Jo-Wilfried Tsonga,33.705681,20181231
5,104871,Jeremy Chardy,31.882272,20181231
8,200282,Alex De Minaur,19.868583,20181231
...,...,...,...,...
182281,214056,Angelica Maria Ochoa Martinez,22.480493,20200309
182308,206306,Koralie Etienne,23.616701,20200113
182395,219358,Kelly Southwood,25.853525,20200210
184405,200712,Katerina Kroupova,46.852841,20201228


In [69]:
#creo due liste, una con l'età attualizzata al 2021 per gli uomini e una per le donne
from datetime import datetime 

lista_uomini=list()
lista_donne=list()

#itero per ogni player con almeno un valore di age. Qui ho eliminato i duplicati, tanto mi basta un record di un player
#per calcolare l'età attualizzata al 2021
for index, row in senza_NAN.iterrows():

    annoTorneo = datetime.strptime(str(row['tourney_date']), '%Y%m%d').strftime('%Y')
    age=int(row['player_age'])
    age_2021= age + (2021- int(annoTorneo))
    
    if row['player_name'] in male_set:
        lista_uomini.append(age_2021)
    elif row['player_name'] in female_set:

        lista_donne.append(age_2021)

    else:
        print(row['player_name']+' non ha valore su sex')
    
    

In [70]:
media_uomini=int(mean(lista_uomini))
media_uomini

26

In [71]:
media_donne=int(mean(lista_donne))
media_donne

23

In [72]:

#funzione che restituisce age di un player, prendendo come argomenti la data del torneo e 
#l'età del player nel 2021 ed assumendo che sia nato il 1 gennaio di 'etaAttualizzata' anni fa
def calcoloAge(dataTorneo, etaAttualizzata):
    
    
    #restituisce null se la data del torneo o l'età sono valori null
    if (dataTorneo=='' or etaAttualizzata==''):
        return '?'
    
    else:
        
        dataTorneo=str(dataTorneo)


        #recupero anno mese e giorno del torneo
        annoTorneo = datetime.strptime(dataTorneo, '%Y%m%d').strftime('%Y')
        meseTorneo=datetime.strptime(dataTorneo, '%Y%m%d').strftime('%m')
        giornoTorneo=datetime.strptime(dataTorneo, '%Y%m%d').strftime('%d')
        
        #calcolo numero di anni
        age_anni=etaAttualizzata-(2021-int(annoTorneo))

        #calcolo la differenza in giorni tra il torneo e il compleanno
        d0 = date(int(annoTorneo), int(meseTorneo), int(giornoTorneo))
        d1 = date(int(annoTorneo), 1, 1)
        delta = d0 - d1
        giorni=delta.days
        #se la differenza in giorni è negativa vuol dire che non ha ancora compiuto gli anni e quindi tolgo un anno da age_anni
        if giorni<0:
            age_anni=age_anni-1
            giorni=-(giorni)
        #calcolo la percentuale sui giorni dell'anno
        percentuale=giorni/365
        
        return age_anni+percentuale
            
            


In [73]:
calcoloAge(20181231,26)

23.997260273972604

In [74]:
solo_NAN

,index,player_id,player_name,player_age,tourney_date
0,5890,209188,Ari Fahresi,NaN,20190914
1,19065,220019,Venia Yeung,NaN,20170724
2,21732,220257,Nour Dib,NaN,20170925
3,36444,220246,Daria Lukyanova,NaN,20161212
4,37538,220247,Shufaa Changawa Ruwa,NaN,20161212
...,...,...,...,...,...
9322,185786,222947,Maya Pitts,NaN,20200224
9323,185940,223451,Chloe Henderson,NaN,20201026
9324,185999,223452,Emma Charney,NaN,20201102
9325,186009,223318,Whitley Pate,NaN,20201102


In [75]:
#itero sui player con valori null su age, riempiendo i null value con le medie in base al sesso, tenendo conto dalla data
#del torneo

for index, row in solo_NAN.iterrows():
    
    index_row=row['index']
    if (tennis.loc[index_row, 'winner_id']==row['player_id']):
        if row['player_name'] in male_set:
            tennis.at[index_row, 'winner_age'] = calcoloAge(row['tourney_date'],media_uomini)
        elif row['player_name'] in female_set:
            tennis.at[index_row, 'winner_age'] = calcoloAge(row['tourney_date'],media_donne)
        else:
            print('errore')
       

    elif (tennis.loc[index_row, 'loser_id']==row['player_id']):
        if row['player_name'] in male_set:
            tennis.at[index_row, 'loser_age'] = calcoloAge(row['tourney_date'],media_uomini)
        elif row['player_name'] in female_set:
            tennis.at[index_row, 'loser_age'] = calcoloAge(row['tourney_date'],media_donne)
        else:
            print('errore')
    else:
        print('errore')

    

       

In [76]:
tennis.loc[5890, 'winner_id']==209188

True

In [77]:
#unisco i player con i loro valori su age tra quelli in winner e quelli in loser
winner=tennis[['winner_id','winner_name','winner_age','tourney_date']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_age':'player_age'})
          
loser=tennis[['loser_id','loser_name','loser_age','tourney_date']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_age':'player_age'})

player=[winner,loser]
player = pd.concat(player)

#creo una tabella con solo valori null
tutti_NAN=player[player['player_age'].isna()]

tutti_NAN.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 163278 to 185303
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   player_id     52 non-null     int64  
 1   player_name   52 non-null     object 
 2   player_age    0 non-null      float64
 3   tourney_date  52 non-null     int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 2.0+ KB


Controllo che applicando una funzione che va a calcolare l'anno di nascita del player, tutti i player abbiano un unico anno di nascita

In [78]:
#funzione che restituisce l'anno di nascita, prendendo come argomenti la data del torneo e 
#l'età del player durante il torneo
def annoNascita(dataTorneo, etaDuranteTorneo):
    
    #restituisce null se la data del torneo o l'età sono valori null
    if (dataTorneo=='' or etaDuranteTorneo==''):
        return '?'
    
    else:
        #recupero anno mese e giorno del torneo
        annoTorneo = datetime.strptime(str(dataTorneo), '%Y%m%d').strftime('%Y')
        meseTorneo=datetime.strptime(str(dataTorneo), '%Y%m%d').strftime('%m')
        giornoTorneo=datetime.strptime(str(dataTorneo), '%Y%m%d').strftime('%d')
        
        #tolgo la parte decimale dell'età del player. Prima da stringa lo trasformo in float e poi da float in int
        anniPlayer=int(float(etaDuranteTorneo))
        #recupero solo la parte decimale che è una frazione rispetto ad un anno di 365 giorni
        frazioneEtaPlayer=float(etaDuranteTorneo) - anniPlayer
        
        #moltiplicando la parte decimale per 365 ottengo il numero di giorni tra il compleanno e la data del torneo
        giorniTraCompleannoTorneo=int(frazioneEtaPlayer*365)
        
        #trasformo in formato date la data del torneo cosi da recuperare poi la data di nascita del player
        dateTorneo=date(int(annoTorneo), int(meseTorneo), int(giornoTorneo)) 
        #recupero la data di nascita utilizzando la funzione timedelta. Vado a sottarre dalla data del 
        #torneo i giorni che separano il compleanno dal torneo. Avere la data di nascita è importante per altrimenti
        #togliendo dalla data del torneo solo gli anni rischio di perdermi un anno se tra il compleanno e la data
        #del torneo c'è di mezzo capodanno. In questo caso il risultato è il giorno ed il mese di nascita del player,
        #ma non ancora l'anno; devo togliere l'età, senza decimali, del player ad inizio torneo
        dateBirthday=dateTorneo - timedelta(days=(giorniTraCompleannoTorneo) )
        
        #dalla data di nascita del player recupero solo l'anno
        result = dateBirthday.strftime("%Y")
        #tolgo dall'anno il valore intero dell'età del player
        result=int(result) -anniPlayer
        
        return result

In [79]:
#per ogni record di tennis vado ad aggiungere 2 colonne con l'anno di nascita del winner e del loser
#il calcolo sfrutta la funzione annoNascita
for index, row in tennis.iterrows():
    
    colonne=['winner','loser']
    for col in colonne:
        if pd.isna(row[col+'_age']):
            tennis.at[index, col+'_data_nascita'] = np.nan
        
        else:
            tennis.at[index, col+'_data_nascita'] = annoNascita(row['tourney_date'],row[col+'_age'])


In [80]:
tennis

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue,winner_data_nascita,loser_data_nascita
0,2019-M020,Brisbane,Hard,32,A,20181231,300,105453,Kei Nishikori,R,178.0,JPN,29.004791,106421,Daniil Medvedev,R,198.0,RUS,22.885695,6-4 3-6 6-2,3,F,124.0,3.0,3.0,77.0,44.0,31.0,17.0,13.0,3.0,6.0,8.0,6.0,100.0,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0,3928,742618.69,1989.0,1996.0
1,2019-M020,Brisbane,Hard,32,A,20181231,299,106421,Daniil Medvedev,R,198.0,RUS,22.885695,104542,Jo-Wilfried Tsonga,R,188.0,FRA,33.705681,7-6(6) 6-2,3,SF,82.0,10.0,1.0,52.0,33.0,28.0,14.0,10.0,0.0,1.0,17.0,2.0,77.0,52.0,36.0,7.0,10.0,10.0,13.0,16.0,1977.0,239.0,200.0,3928,742618.69,1996.0,1985.0
2,2019-M020,Brisbane,Hard,32,A,20181231,298,105453,Kei Nishikori,R,178.0,JPN,29.004791,104871,Jeremy Chardy,R,188.0,FRA,31.882272,6-2 6-2,3,SF,66.0,2.0,2.0,47.0,33.0,26.0,9.0,8.0,2.0,2.0,10.0,3.0,46.0,27.0,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0,3928,742618.69,1989.0,1987.0
3,2019-M020,Brisbane,Hard,32,A,20181231,297,104542,Jo-Wilfried Tsonga,R,188.0,FRA,33.705681,200282,Alex De Minaur,R,183.0,AUS,19.868583,6-4 7-6(2),3,QF,106.0,12.0,2.0,68.0,43.0,34.0,15.0,11.0,4.0,5.0,1.0,2.0,81.0,60.0,38.0,9.0,11.0,4.0,6.0,239.0,200.0,31.0,1298.0,3928,742618.69,1985.0,1999.0
4,2019-M020,Brisbane,Hard,32,A,20181231,296,106421,Daniil Medvedev,R,198.0,RUS,22.885695,105683,Milos Raonic,R,196.0,CAN,28.010951,6-7(2) 6-3 6-4,3,QF,129.0,12.0,3.0,105.0,68.0,48.0,25.0,16.0,8.0,8.0,29.0,5.0,94.0,56.0,46.0,19.0,15.0,2.0,4.0,16.0,1977.0,18.0,1855.0,3928,742618.69,1996.0,1990.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186068,2020-W-ITF-USA-48A-2020,W25 Orlando FL,Hard,32,25,20201109,204,215794,Hurricane Tyra Black,R,172.0,USA,19.690623,222081,Eleana Yu,R,172.0,USA,16.002738,7-6(1) 6-1,3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,598.0,57.0,1216.0,6.0,2312,471620.95,2001.0,2004.0
186069,2020-W-ITF-USA-48A-2020,W25 Orlando FL,Hard,32,25,20201109,205,210754,Vivian Heisen,U,172.0,GER,26.869268,220723,Jessica Livianu,R,172.0,USA,22.713210,2-6 6-2 10-3,3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,429.0,102.0,729.0,35.0,2312,471620.95,1993.0,1998.0
186070,2020-W-ITF-USA-48A-2020,W25 Orlando FL,Hard,32,25,20201109,206,211874,Victoria Rodriguez,U,172.0,MEX,25.552361,202478,Tara Moore,R,172.0,GBR,28.260096,6-2 7-6(8),3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,643.0,48.0,454.0,93.0,2312,471620.95,1995.0,1992.0
186071,2020-W-ITF-USA-48A-2020,W25 Orlando FL,Hard,32,25,20201109,207,213789,Katarina Jokic,U,172.0,BIH,22.559890,205922,Chanelle Van Nguyen,U,172.0,USA,26.806297,6-4 6-2,3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,923.0,17.0,627.0,51.0,2312,471620.95,1998.0,1994.0


In [81]:
#vado a vedere quanti player hanno valori diversi su per l'anno di nascita
#se hanno valori diversi vuol dire che l'attrbuto Age ha dati non corretti su alcuni record
winner=tennis[['winner_id','winner_name','winner_age','tourney_date','winner_data_nascita']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_age':'player_age'
                              ,'winner_data_nascita':'data_nascita'})
          
loser=tennis[['loser_id','loser_name','loser_age','tourney_date','loser_data_nascita']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_age':'player_age',
                           'loser_data_nascita':'data_nascita'})

player=[winner,loser]
player = pd.concat(player)
player=player.dropna()
player=player.drop_duplicates(subset=['player_id', 'player_name','data_nascita'])

player_Error_age=player.groupby(['player_name'])['player_name'].count()
player_Error_age=player_Error_age[player_Error_age>1]


print(player_Error_age)

print('')
print('Totale nomi con anno nascita diverso = ',end='')
print(len(player_Error_age))

player_name
Adrian Menendez Maceiras    2
Adrien Bossel               2
Alexey Vatutin              2
Andrea Arnaboldi            2
Andrew Paulson              2
Antoine Hoang               2
Aslan Karatsev              2
Blaz Rola                   2
Bojana Jovanovski           2
Botic van de Zandschulp     2
Brydan Klein                2
Camilo Andres Erazo         2
Cedrik Marcel Stebe         2
Dane Webb                   2
Daniel Altmaier             2
Dmitry Popko                2
Elliot Benchetrit           2
Eric Ahren Moonga           2
Ernests Gulbis              2
Evgeny Karlovskiy           2
Facundo Mena                2
Gilles Simon                2
Giulio Zeppieri             2
Hugo Gaston                 2
Jacopo Berrettini           2
Jiri Lehecka                2
Joao Monteiro               2
Jose Daniel Bendeck         2
Jozef Kovalik               2
Katie Volynets              2
Kei Nishikori               2
Kirill Komarov              2
Kuan Yi Lee                 

In [82]:
##stampo tutti i player con diverso valori su anno nascita. Sono 58.
# inserisco in un dizionario il primo valore su anno di nascita che trovo, con chiave il nome del player
dizionario=dict()
for index, value in player_Error_age.items():
    
    print(str(index) + ' --> ',end='')

    rows=player.loc[player['player_name']==index]
    first=True
    for index_player, row_player in rows.iterrows():
        if first:
            print(str(row_player['data_nascita']) + ' - ',end='')
            first=False
            dizionario[index]=row_player['data_nascita']
        else:
            print(row_player['data_nascita'])

Adrian Menendez Maceiras --> 1985.0 - 1986.0
Adrien Bossel --> 1986.0 - 1987.0
Alexey Vatutin --> 1992.0 - 1993.0
Andrea Arnaboldi --> 1987.0 - 1988.0
Andrew Paulson --> 2002.0 - 2001.0
Antoine Hoang --> 1995.0 - 1996.0
Aslan Karatsev --> 1993.0 - 1994.0
Blaz Rola --> 1990.0 - 1991.0
Bojana Jovanovski --> 1991.0 - 1992.0
Botic van de Zandschulp --> 1995.0 - 1996.0
Brydan Klein --> 1990.0 - 1989.0
Camilo Andres Erazo --> 1996.0 - 1997.0
Cedrik Marcel Stebe --> 1990.0 - 1991.0
Dane Webb --> 1992.0 - 1993.0
Daniel Altmaier --> 1998.0 - 1999.0
Dmitry Popko --> 1996.0 - 1997.0
Elliot Benchetrit --> 1998.0 - 1999.0
Eric Ahren Moonga --> 1995.0 - 1996.0
Ernests Gulbis --> 1988.0 - 1989.0
Evgeny Karlovskiy --> 1994.0 - 1995.0
Facundo Mena --> 1992.0 - 1993.0
Gilles Simon --> 1984.0 - 1985.0
Giulio Zeppieri --> 2001.0 - 2002.0
Hugo Gaston --> 2000.0 - 2001.0
Jacopo Berrettini --> 1998.0 - 1999.0
Jiri Lehecka --> 2002.0 - 2001.0
Joao Monteiro --> 1993.0 - 1994.0
Jose Daniel Bendeck --> 1990.0 - 

In [83]:
player.loc[player['player_name']=='Aslan Karatsev']

,player_id,player_name,player_age,tourney_date,data_nascita
55419,106234,Aslan Karatsev,22.715948,20160523,1993.0
90624,106234,Aslan Karatsev,26.529774,20200817,1994.0


In [84]:
#itero su tennis, quando trovo un nome del player presente nel dizionario con valore di anno di nascita
#diverso da quello scelto nel dizionario setto il valore su Age a Null.
#in questo modo garantisco che da Age sia calcolato sempre lo stesso anno di nascita a prescinde dall'ordine in cui leggo i dati
for index, row in tennis.iterrows():
    
    colonne=['winner','loser']
    for col in colonne:
        if (row[col+'_name']) in dizionario:
            if row[col+'_data_nascita'] != dizionario[row[col+'_name']]:
                tennis.at[index, col+'_age'] = np.nan


In [85]:
#controllo che non ci siano più errori
winner=tennis[['winner_id','winner_name','winner_age','tourney_date','winner_data_nascita']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_age':'player_age'
                              ,'winner_data_nascita':'data_nascita'})
          
loser=tennis[['loser_id','loser_name','loser_age','tourney_date','loser_data_nascita']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_age':'player_age',
                           'loser_data_nascita':'data_nascita'})

player=[winner,loser]
player = pd.concat(player)
player=player.dropna()
player=player.drop_duplicates(subset=['player_id', 'player_name','data_nascita'])

player_Error_age=player.groupby(['player_name'])['player_name'].count()
player_Error_age=player_Error_age[player_Error_age>1]


print(player_Error_age)

print('')
print('Totale nomi con anno nascita diverso = ',end='')
print(len(player_Error_age))

Series([], Name: player_name, dtype: int64)

Totale nomi con anno nascita diverso = 0


In [86]:
#elimino le colonne con la data di nascita create
tennis=tennis.drop(['winner_data_nascita', 'loser_data_nascita'], axis=1)

## NULL VALUE IN SCORE

In [87]:
tennis['score'].value_counts()

6-3 6-4               5367
6-3 6-2               5252
6-4 6-4               5193
6-2 6-2               5089
6-3 6-3               4836
                      ... 
4-6 7-6(7) 2-3 RET       1
7-5 0-2 RET              1
6-7(1) 6-1 0-0 RET       1
3-6 2-2 RET              1
7-6(4) 2-6 10-5          1
Name: score, Length: 9705, dtype: int64

In [88]:
# I NULL VALUE SU SCORE SONO RIEMPITI CON LA MODA
moda=tennis['score'].mode()
moda[0]

'6-3 6-4'

In [89]:
tennis['score'].fillna(moda[0],inplace = True)

## NULL VALUE IN RANK

In [90]:
##per riempire i valori su winner_rank e su loser_rank, è stata calcolata la media per ogni specifico player. 
#I valori mancanti per un player sono stati riempiti con la sua media su rank. Se un player ha solo valori null
#su rank allora gli viene impostata la media di tutte le medie

In [91]:
tennis['winner_rank'].value_counts()

1.0       514
2.0       459
5.0       451
3.0       438
6.0       422
         ... 
1631.0      1
1772.0      1
1956.0      1
1545.0      1
1843.0      1
Name: winner_rank, Length: 1623, dtype: int64

In [92]:
winner=tennis[['winner_id','winner_name','winner_rank']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_rank':'player_rank'})
          
loser=tennis[['loser_id','loser_name','loser_rank']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_rank':'player_rank'})

player=[winner,loser]
player = pd.concat(player)
player=player.sort_values(by=['player_name'])
player


,player_id,player_name,player_rank
167337,241720,Aada Inna,NaN
155322,222088,Aalisha Alexis,NaN
155349,222088,Aalisha Alexis,NaN
140198,222673,Aaliya Ebrahim,NaN
140250,222673,Aaliya Ebrahim,NaN
...,...,...,...
170479,242443,Zuzanna Bednarz,NaN
152014,222154,Zuzanna Szczepanska,NaN
170519,222154,Zuzanna Szczepanska,NaN
151955,222154,Zuzanna Szczepanska,NaN


In [93]:
#elimino le righe con null value
player_senza_NAN=player.dropna()
player_senza_NAN

,player_id,player_name,player_rank
76205,126608,Aaro Pollanen,1559.0
103762,132124,Aaron Addison,1037.0
71687,132124,Aaron Addison,1037.0
89564,132124,Aaron Addison,1055.0
72918,132124,Aaron Addison,1254.0
...,...,...,...
47274,202636,Zuzana Zlochova,455.0
47227,202636,Zuzana Zlochova,445.0
47123,202636,Zuzana Zlochova,544.0
38361,202636,Zuzana Zlochova,427.0


In [94]:
#faccio la media di rank su ogni player
media_rank=player_senza_NAN.groupby(['player_name'])['player_rank'].mean()
media_rank


player_name
Aaro Pollanen               1559.000000
Aaron Addison               1130.900000
Abbie Myers                  474.929936
Abdullah Maqdas             1298.000000
Abduvoris Saitmukhamedov    2061.000000
                               ...     
Zoziya Kardava              1106.833333
Zsombor Piros                452.593023
Zuzana Luknarova             831.000000
Zuzana Zalabska              858.000000
Zuzana Zlochova              460.664804
Name: player_rank, Length: 4447, dtype: float64

In [95]:
#calcolo la media per i player che non hanno mai il rank
media_player_NotIn_media_rank=float(int(media_rank.mean()))
media_player_NotIn_media_rank

825.0

In [96]:
#itero su tennis, riempiendo i null value con le medie di ogni player in base al rank. 
#Ricerco il valore sui media_rank
for index, row in tennis.iterrows():
    
    colonne=['winner','loser']
    for col in colonne:
        if pd.isna(row[col+'_rank']):
            if row[col+'_name'] in media_rank:
                rank=float(int(media_rank[row[col+'_name']]))
        
            else:
                rank=media_player_NotIn_media_rank

            tennis.at[index, col+'_rank'] = rank
       


In [97]:
tennis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185758 entries, 0 to 186072
Data columns (total 47 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          185758 non-null  object 
 1   tourney_name        185758 non-null  object 
 2   surface             185596 non-null  object 
 3   draw_size           185758 non-null  int64  
 4   tourney_level       185758 non-null  object 
 5   tourney_date        185758 non-null  int64  
 6   match_num           185758 non-null  int64  
 7   winner_id           185758 non-null  int64  
 8   winner_name         185758 non-null  object 
 9   winner_hand         185758 non-null  object 
 10  winner_ht           185758 non-null  float64
 11  winner_ioc          185758 non-null  object 
 12  winner_age          185518 non-null  float64
 13  loser_id            185758 non-null  int64  
 14  loser_name          185758 non-null  object 
 15  loser_hand          185758 non-nul

## NULL VALUE IN RANK_POINTS

In [98]:
##per riempire i valori su winner_rank_points e su loser_rank_points, è stata calcolata la media per ogni specifico player. 
#I valori mancanti per un player sono stati riempiti con la sua media su rank_points. Se un player ha solo valori null
#su rank_points allora gli viene impostata la media di tutte le medie

In [99]:
tennis['winner_rank_points'].value_counts()

4.0       1376
8.0       1300
7.0       1264
3.0       1245
5.0       1232
          ... 
1591.0       1
2274.0       1
1843.0       1
4230.0       1
5650.0       1
Name: winner_rank_points, Length: 3144, dtype: int64

In [100]:
winner=tennis[['winner_id','winner_name','winner_rank_points']]
winner=winner.rename(columns={'winner_id': 'player_id','winner_name':'player_name','winner_rank_points':'player_rank_points'})
          
loser=tennis[['loser_id','loser_name','loser_rank_points']]
loser=loser.rename(columns={'loser_id': 'player_id','loser_name':'player_name','loser_rank_points':'player_rank_points'})

player=[winner,loser]
player = pd.concat(player)
player=player.sort_values(by=['player_name'])
player

,player_id,player_name,player_rank_points
167337,241720,Aada Inna,NaN
155322,222088,Aalisha Alexis,NaN
155349,222088,Aalisha Alexis,NaN
140198,222673,Aaliya Ebrahim,NaN
140250,222673,Aaliya Ebrahim,NaN
...,...,...,...
170479,242443,Zuzanna Bednarz,NaN
152014,222154,Zuzanna Szczepanska,NaN
170519,222154,Zuzanna Szczepanska,NaN
151955,222154,Zuzanna Szczepanska,NaN


In [101]:
#elimino le righe con null value
player_senza_NAN=player.dropna()
player_senza_NAN

,player_id,player_name,player_rank_points
76205,126608,Aaro Pollanen,2.0
103762,132124,Aaron Addison,8.0
71687,132124,Aaron Addison,10.0
89564,132124,Aaron Addison,8.0
72918,132124,Aaron Addison,6.0
...,...,...,...
47274,202636,Zuzana Zlochova,71.0
47227,202636,Zuzana Zlochova,72.0
47123,202636,Zuzana Zlochova,47.0
38361,202636,Zuzana Zlochova,78.0


In [102]:
#faccio la media di rank su ogni player
media_rank_points=player_senza_NAN.groupby(['player_name'])['player_rank_points'].mean()
media_rank_points

player_name
Aaro Pollanen                2.000000
Aaron Addison                7.700000
Abbie Myers                 98.828025
Abdullah Maqdas              4.000000
Abduvoris Saitmukhamedov     1.000000
                              ...    
Zoziya Kardava               9.648148
Zsombor Piros               74.174419
Zuzana Luknarova            17.000000
Zuzana Zalabska             14.000000
Zuzana Zlochova             87.782123
Name: player_rank_points, Length: 4446, dtype: float64

In [103]:
#calcolo la media per i player che non hanno mai il rank
media_player_NotIn_media_rank_points=float(int(media_rank_points.mean()))
media_player_NotIn_media_rank_points

142.0

In [104]:
#itero su tennis, riempiendo i null value con le medie di ogni player in base al rank_points. Ricerco il valore su
#media_rank_points

for index, row in tennis.iterrows():
    
    colonne=['winner','loser']
    for col in colonne:
        if pd.isna(row[col+'_rank_points']):
            if row[col+'_name'] in media_rank_points:
                rank_points=float(int(media_rank_points[row[col+'_name']]))
        
            else:
                rank_points=media_player_NotIn_media_rank_points

            tennis.at[index, col+'_rank_points'] = rank_points

## PROBLEMI CON I TORNEI

## Problema con il livello del torneo

Il problema con i valori di level è che per uno stesso torneo id sono associati spesso due livelli differenti del torneo. Questo analizzando i dati è probabilmente causato da tornei dove si giocano in contemporanea partite maschili e femminili, che presentano alcune volte una nomenclatura del livello differente. Per risolvare questo problema e lasciare il torneo id come chiave univoca, abbiamo deciso di sostituire l'id del torneo con la concatenzazione dell'id del torneo + _ + livello del torneo

In [105]:
tennis['tourney_level'].value_counts()

15     45804
C      40527
25     30743
A      11678
10     11372
G       9981
I       8452
60      6517
P       5944
M       3877
100     2685
D       2373
80      1909
50      1488
PM      1478
W        436
F        211
75       155
O        128
Name: tourney_level, dtype: int64

In [106]:
#qui vediamo come per 57 tornei id siano associati due valori diversi in level
torneo=tennis[['tourney_id','tourney_name','tourney_level']]
torneo=torneo.drop_duplicates(subset=['tourney_id', 'tourney_level'])

torneo_Error=torneo.groupby(['tourney_id'])['tourney_level'].count()
torneo_Error=torneo_Error[torneo_Error>1]

torneo_Error.sort_values()

tourney_id
2016-M001    2
2018-M007    2
2018-M009    2
2018-M010    2
2018-M014    2
2018-M015    2
2018-M016    2
2018-M020    2
2018-M021    2
2018-M024    2
2018-M035    2
2019-M001    2
2019-M004    2
2019-M006    2
2019-M007    2
2019-M009    2
2019-M010    2
2019-M014    2
2019-M015    2
2019-M016    2
2019-M020    2
2019-M021    2
2019-M024    2
2019-M035    2
2020-M004    2
2020-M009    2
2018-M006    2
2020-M024    2
2018-M004    2
2017-M035    2
2016-M004    2
2016-M006    2
2016-M007    2
2016-M009    2
2016-M010    2
2016-M014    2
2016-M015    2
2016-M020    2
2016-M021    2
2016-M024    2
2016-M035    2
2016-M052    2
2016-O16     2
2017-M001    2
2017-M004    2
2017-M006    2
2017-M007    2
2017-M009    2
2017-M010    2
2017-M014    2
2017-M015    2
2017-M016    2
2017-M020    2
2017-M021    2
2017-M024    2
2018-M001    2
2020-M056    2
Name: tourney_level, dtype: int64

In [107]:
##stampo tutti gli id dei tornei e i nomi con diversi valori su level

for index, value in torneo_Error.items():

    print(str(index) + ' --> ',end='')

    rows=torneo.loc[torneo['tourney_id']==index]
    first=True
    for index_torneo, row_torneo in rows.iterrows():
        if first:
            print(str(row_torneo['tourney_level']) + ' - ',end='')
            first=False
        else:
            print(row_torneo['tourney_level'])
            

2016-M001 --> P - A
2016-M004 --> I - A
2016-M006 --> P - M
2016-M007 --> P - M
2016-M009 --> P - M
2016-M010 --> I - A
2016-M014 --> P - A
2016-M015 --> P - A
2016-M020 --> P - A
2016-M021 --> P - M
2016-M024 --> P - M
2016-M035 --> I - A
2016-M052 --> I - A
2016-O16 --> O - A
2017-M001 --> I - A
2017-M004 --> I - A
2017-M006 --> PM - M
2017-M007 --> PM - M
2017-M009 --> P - M
2017-M010 --> I - A
2017-M014 --> I - A
2017-M015 --> PM - A
2017-M016 --> P - A
2017-M020 --> I - A
2017-M021 --> PM - M
2017-M024 --> P - M
2017-M035 --> I - A
2018-M001 --> A - P
2018-M004 --> A - I
2018-M006 --> M - PM
2018-M007 --> M - PM
2018-M009 --> M - P
2018-M010 --> A - I
2018-M014 --> A - P
2018-M015 --> A - PM
2018-M016 --> A - P
2018-M020 --> A - P
2018-M021 --> M - PM
2018-M024 --> M - PM
2018-M035 --> A - I
2019-M001 --> A - P
2019-M004 --> A - I
2019-M006 --> M - PM
2019-M007 --> M - PM
2019-M009 --> M - P
2019-M010 --> A - I
2019-M014 --> A - I
2019-M015 --> A - PM
2019-M016 --> A - P
2019-M020

In [108]:
#eseguo la concatenazione di tourney_id e tourney_level separati da _
#aggiorno il valore di tourney_id
tourney_id_level=tennis['tourney_id'].astype(str)+'_'+tennis['tourney_level'].astype(str)
tennis['tourney_id']=tourney_id_level

In [109]:
#controllo che non ci siano più problemi con il torneo id
torneo=tennis[['tourney_id','tourney_name','tourney_level']]
torneo=torneo.drop_duplicates(subset=['tourney_id', 'tourney_name','tourney_level'])

torneo_Error=torneo.groupby(['tourney_id','tourney_name'])['tourney_level'].count()
torneo_Error=torneo_Error[torneo_Error>1]

torneo_Error.sort_values()

Series([], Name: tourney_level, dtype: int64)

In [110]:
##2 tornei presentano stesso torneo id ma diverso nome. 1 è causato da un errore di 
#digitalizzazione del dato, mentre l'altro presenta rappresenta l'id del torneo di qualificazione
#per gli australian open e cambia nome in base a femminile o maschile; si è deciso di
#lasciare solo la parte di nome comune


In [111]:
#creo un dataframe con id del torneo e nome del torneo
torneo=tennis[['tourney_id','tourney_name']]
#elimino le righe duplicate
torneo=torneo.drop_duplicates(subset=['tourney_id', 'tourney_name'])
#faccio una groupby sull'id per vedere quanti nomi diversi sono associati ad ogni id. Elimino i record in cui ad un id
#corrisponde solo un nome
torneo_Error=torneo.groupby(['tourney_id'])['tourney_name'].count()
torneo_Error=torneo_Error[torneo_Error>1]

print(torneo_Error)

print('')
print('Totale tornei_id con nomi diversi = ',end='')
print(len(torneo_Error))

tourney_id
2019-560_G    2
2021-581_G    2
Name: tourney_name, dtype: int64

Totale tornei_id con nomi diversi = 2


In [112]:
##stampo tutti gli id dei tornei con diverso valori su name
for index, value in torneo_Error.items():
    
    print(str(index) + ' --> ',end='')

    rows=torneo.loc[torneo['tourney_id']==index]
    first=True
    for index_torneo, row_torneo in rows.iterrows():
        if first:
            print(str(row_torneo['tourney_name']) + ' - ',end='')
            first=False
        else:
            print(row_torneo['tourney_name'])

2019-560_G --> US Open - Us Open
2021-581_G --> Doha Aus Open Qualies - Dubai Aus Open Qualies


In [113]:
#modifico i nomi dei 2 tornei id 'manualmente'

t=tennis.loc[tennis['tourney_name']=='Us Open']
for index, row in t.iterrows():
    tennis.at[index, 'tourney_name'] = "US Open"
    
t=tennis.loc[tennis['tourney_name']=='Doha Aus Open Qualies']
for index, row in t.iterrows():
    tennis.at[index, 'tourney_name'] = "Aus Open Qualies"

t=tennis.loc[tennis['tourney_name']=='Dubai Aus Open Qualies']
for index, row in t.iterrows():
    tennis.at[index, 'tourney_name'] = "Aus Open Qualies"
    


In [114]:
torneo=tennis[['tourney_id','tourney_name']]
torneo=torneo.drop_duplicates(subset=['tourney_id', 'tourney_name'])

torneo_Error=torneo.groupby(['tourney_id'])['tourney_name'].count()
torneo_Error=torneo_Error[torneo_Error>1]

##stampo tutti gli id dei tornei con diverso valori su name
for index, value in torneo_Error.items():
    
    print(str(index) + ' --> ',end='')

    rows=torneo.loc[torneo['tourney_id']==index]
    first=True
    for index_torneo, row_torneo in rows.iterrows():
        if first:
            print(str(row_torneo['tourney_name']) + ' - ',end='')
            first=False
        else:
            print(row_torneo['tourney_name'])


In [115]:
# modifico i nomi degli altri 22 tornei_id in cui appare la parola Master
#itero su torneo_Error, cioè la Series con gli id dei tornei da modificare come index
for index, value in torneo_Error.items():
    #seleziono solo le righe del dataframe tennis con id torneo == index
    rows=tennis.loc[tennis['tourney_id']==index]
    for index_tennis, row_tennis in rows.iterrows():
        #sostituisco la parola Master, se presente, con niente
        new_Name=row_tennis['tourney_name'].replace('Masters','').strip()
        tennis.at[index_tennis, 'tourney_name'] = new_Name


In [116]:
#controllo se ci sono ancora tornei id con nomi diversi
torneo=tennis[['tourney_id','tourney_name']]
torneo=torneo.drop_duplicates(subset=['tourney_id', 'tourney_name'])

torneo_Error=torneo.groupby(['tourney_id'])['tourney_name'].count()
torneo_Error=torneo_Error[torneo_Error>1]

print(torneo_Error)

print('')
print('Totale tornei_id con nomi diversi = ',end='')
print(len(torneo_Error)) 

Series([], Name: tourney_name, dtype: int64)

Totale tornei_id con nomi diversi = 0


In [117]:
tennis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185758 entries, 0 to 186072
Data columns (total 47 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          185758 non-null  object 
 1   tourney_name        185758 non-null  object 
 2   surface             185596 non-null  object 
 3   draw_size           185758 non-null  int64  
 4   tourney_level       185758 non-null  object 
 5   tourney_date        185758 non-null  int64  
 6   match_num           185758 non-null  int64  
 7   winner_id           185758 non-null  int64  
 8   winner_name         185758 non-null  object 
 9   winner_hand         185758 non-null  object 
 10  winner_ht           185758 non-null  float64
 11  winner_ioc          185758 non-null  object 
 12  winner_age          185518 non-null  float64
 13  loser_id            185758 non-null  int64  
 14  loser_name          185758 non-null  object 
 15  loser_hand          185758 non-nul

## problema miss value surface

In [118]:
# non sono presenti tornei con stesso nome ma diversa superficie. I missing value sono stati riempiti con la moda

In [119]:
torneo=tennis[['tourney_id','surface']]
torneo=torneo.drop_duplicates(subset=['tourney_id', 'surface'])

torneo_Error=torneo.groupby(['tourney_id'])['surface'].count()
torneo_Error=torneo_Error[torneo_Error>1]

torneo_Error.sort_values()

Series([], Name: surface, dtype: int64)

In [120]:
tornei_solo_NAN=torneo[torneo['surface'].isna()]

tornei_solo_NAN

,tourney_id,surface
86167,2017-W-FC-2017-G2-AM-A-M-ECU-CUB-01_D,NaN
86169,2017-W-FC-2017-G2-AM-A-M-ECU-URU-01_D,NaN
86171,2017-W-FC-2017-G2-AM-A-M-URU-CUB-01_D,NaN
86173,2017-W-FC-2017-G2-AM-B-M-DOM-BAR-01_D,NaN
86175,2017-W-FC-2017-G2-AM-B-M-GUA-BAR-01_D,NaN
...,...,...
162939,2017-M-DC-2017-G2-EPA-M-MON-SLO-01_D,NaN
162943,2017-M-DC-2017-G2-EPA-M-NOR-LAT-01_D,NaN
162955,2017-M-DC-2017-G2-EPA-M-TUR-CYP-01_D,NaN
162959,2017-M-DC-2017-G2-AO-M-VIE-HKG-01_D,NaN


In [121]:
for index, row in tornei_solo_NAN.iterrows():
    print(row['tourney_id'])


2017-W-FC-2017-G2-AM-A-M-ECU-CUB-01_D
2017-W-FC-2017-G2-AM-A-M-ECU-URU-01_D
2017-W-FC-2017-G2-AM-A-M-URU-CUB-01_D
2017-W-FC-2017-G2-AM-B-M-DOM-BAR-01_D
2017-W-FC-2017-G2-AM-B-M-GUA-BAR-01_D
2017-W-FC-2017-G2-AM-B-M-GUA-DOM-01_D
2017-W-FC-2017-G2-AM-C-M-PUR-HON-01_D
2017-W-FC-2017-G2-AM-C-M-PUR-TTO-01_D
2017-W-FC-2017-G2-AM-C-M-TTO-HON-01_D
2017-W-FC-2017-G2-AM-D-M-BAH-PAN-01_D
2017-W-FC-2017-G2-AM-D-M-CRC-BAH-01_D
2017-W-FC-2017-G2-AM-D-M-CRC-PAN-01_D
2017-W-FC-2017-G2-AM-D-M-PER-BAH-01_D
2017-W-FC-2017-G2-AM-D-M-PER-CRC-01_D
2017-W-FC-2017-G2-AM-D-M-PER-PAN-01_D
2017-W-FC-2017-G2-AM-PP5-8-CUB-TTO-01_D
2017-W-FC-2017-G2-AM-PP5-8-DOM-CRC-01_D
2017-W-FC-2017-G2-AM-PP9-12-BAR-BAH-01_D
2017-W-FC-2017-G2-AM-PP9-12-URU-HON-01_D
2017-W-FC-2017-G2-AM-PPO-ECU-PUR-01_D
2017-W-FC-2017-G2-AM-PPO-GUA-PER-01_D
2016-W-FC-2016-G2-AM-A-M-CHI-CRC-01_D
2016-W-FC-2016-G2-AM-A-M-CHI-HON-01_D
2016-W-FC-2016-G2-AM-A-M-CRC-HON-01_D
2016-W-FC-2016-G2-AM-A-M-VEN-CHI-01_D
2016-W-FC-2016-G2-AM-A-M-VEN-CRC-01_D
20

In [122]:
tennis['surface'].value_counts()

Hard      94931
Clay      81012
Grass      6600
Carpet     3053
Name: surface, dtype: int64

In [123]:
moda=tennis['surface'].mode()
moda=moda[0]

In [124]:
tennis['surface'].fillna(moda, inplace=True)

## PROBLEMI CON DRAW SIZE

In [125]:
#10 tornei id diversi presentano valori diversi su draw_size. I valori diversi sono stati resi uguali inserendo la moda
# su draw size del torneo

In [126]:
tennis['draw_size'].value_counts()

32     150950
64      17479
128     12426
4        1355
2        1018
48        586
30        435
31        378
28        174
29        167
8         151
24        140
12         90
25         72
27         52
54         51
20         38
15         31
18         30
16         30
19         30
10         28
26         25
23         22
Name: draw_size, dtype: int64

In [127]:
torneo=tennis[['tourney_id','draw_size']]
torneo=torneo.drop_duplicates(subset=['tourney_id', 'draw_size'])

torneo_Error=torneo.groupby(['tourney_id'])['draw_size'].count()
torneo_Error=torneo_Error[torneo_Error>1]

print(torneo_Error)

print('')
print('Totale tornei_id con nomi diversi = ',end='')
print(len(torneo_Error))

Series([], Name: draw_size, dtype: int64)

Totale tornei_id con nomi diversi = 0


## la data del torneo e' ok

In [128]:
torneo=tennis[['tourney_id','tourney_name','tourney_date']]
torneo=torneo.drop_duplicates(subset=['tourney_id', 'tourney_name','tourney_date'])

torneo_Error=torneo.groupby(['tourney_id'])['tourney_date'].count()
torneo_Error=torneo_Error[torneo_Error>1]

torneo_Error.sort_values()

Series([], Name: tourney_date, dtype: int64)

## PROBLEMA CON MATCH NUMBER

#sono presenti 21261 valori distinti con più righe uguali su id torneo e match number. Modifico tutti i valori di match_number incrementando ogni volta di 1 per lo stesso torneo, ripartendo da zero per ogni nuovo torneo

In [129]:
torneo=tennis[['tourney_id','match_num']]

torneo_Error=torneo.groupby(['tourney_id','match_num'])['match_num'].count()
torneo_Error=torneo_Error[torneo_Error>1]

torneo_Error.sort_values()

tourney_id                  match_num
2019-W-ITF-TUN-47A-2019_15  14           2
2020-W-ITF-GER-01A-2020_15  104          2
                            103          2
                            102          2
                            101          2
                                        ..
2019-540_G                  116          4
                            117          4
                            118          4
                            120          4
2020-580_G                  106          4
Name: match_num, Length: 21261, dtype: int64

In [130]:
#fa una groupby in base all'id del torneo e per ogni record con stesso torneo id va a 
#impostare un valore su match che parte da zero viene incrementato ogni nuova row
tennis['match_num'] = tennis.groupby('tourney_id').cumcount()

In [131]:
torneo=tennis[['tourney_id','match_num']]

torneo_Error=torneo.groupby(['tourney_id','match_num'])['match_num'].count()
torneo_Error=torneo_Error[torneo_Error>1]

torneo_Error.sort_values()

Series([], Name: match_num, dtype: int64)

In [184]:
tennis

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
0,2019-M020_A,Brisbane,Hard,32,A,20181231,0,105453,Kei Nishikori,R,178.0,JPN,29.004791,106421,Daniil Medvedev,R,198.0,RUS,22.885695,6-4 3-6 6-2,3,F,124.0,3.0,3.0,77.0,44.0,31.0,17.0,13.0,3.0,6.0,8.0,6.0,100.0,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0,3928,742618.69
1,2019-M020_A,Brisbane,Hard,32,A,20181231,1,106421,Daniil Medvedev,R,198.0,RUS,22.885695,104542,Jo-Wilfried Tsonga,R,188.0,FRA,33.705681,7-6(6) 6-2,3,SF,82.0,10.0,1.0,52.0,33.0,28.0,14.0,10.0,0.0,1.0,17.0,2.0,77.0,52.0,36.0,7.0,10.0,10.0,13.0,16.0,1977.0,239.0,200.0,3928,742618.69
2,2019-M020_A,Brisbane,Hard,32,A,20181231,2,105453,Kei Nishikori,R,178.0,JPN,29.004791,104871,Jeremy Chardy,R,188.0,FRA,31.882272,6-2 6-2,3,SF,66.0,2.0,2.0,47.0,33.0,26.0,9.0,8.0,2.0,2.0,10.0,3.0,46.0,27.0,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0,3928,742618.69
3,2019-M020_A,Brisbane,Hard,32,A,20181231,3,104542,Jo-Wilfried Tsonga,R,188.0,FRA,33.705681,200282,Alex De Minaur,R,183.0,AUS,19.868583,6-4 7-6(2),3,QF,106.0,12.0,2.0,68.0,43.0,34.0,15.0,11.0,4.0,5.0,1.0,2.0,81.0,60.0,38.0,9.0,11.0,4.0,6.0,239.0,200.0,31.0,1298.0,3928,742618.69
4,2019-M020_A,Brisbane,Hard,32,A,20181231,4,106421,Daniil Medvedev,R,198.0,RUS,22.885695,105683,Milos Raonic,R,196.0,CAN,28.010951,6-7(2) 6-3 6-4,3,QF,129.0,12.0,3.0,105.0,68.0,48.0,25.0,16.0,8.0,8.0,29.0,5.0,94.0,56.0,46.0,19.0,15.0,2.0,4.0,16.0,1977.0,18.0,1855.0,3928,742618.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186068,2020-W-ITF-USA-48A-2020_25,W25 Orlando FL,Hard,32,25,20201109,50,215794,Hurricane Tyra Black,R,172.0,USA,19.690623,222081,Eleana Yu,R,172.0,USA,16.002738,7-6(1) 6-1,3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,598.0,57.0,1216.0,6.0,2312,471620.95
186069,2020-W-ITF-USA-48A-2020_25,W25 Orlando FL,Hard,32,25,20201109,51,210754,Vivian Heisen,U,172.0,GER,26.869268,220723,Jessica Livianu,R,172.0,USA,22.713210,2-6 6-2 10-3,3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,429.0,102.0,729.0,35.0,2312,471620.95
186070,2020-W-ITF-USA-48A-2020_25,W25 Orlando FL,Hard,32,25,20201109,52,211874,Victoria Rodriguez,U,172.0,MEX,25.552361,202478,Tara Moore,R,172.0,GBR,28.260096,6-2 7-6(8),3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,643.0,48.0,454.0,93.0,2312,471620.95
186071,2020-W-ITF-USA-48A-2020_25,W25 Orlando FL,Hard,32,25,20201109,53,213789,Katarina Jokic,U,172.0,BIH,22.559890,205922,Chanelle Van Nguyen,U,172.0,USA,26.806297,6-4 6-2,3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,923.0,17.0,627.0,51.0,2312,471620.95


In [132]:
tennis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185758 entries, 0 to 186072
Data columns (total 47 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          185758 non-null  object 
 1   tourney_name        185758 non-null  object 
 2   surface             185758 non-null  object 
 3   draw_size           185758 non-null  int64  
 4   tourney_level       185758 non-null  object 
 5   tourney_date        185758 non-null  int64  
 6   match_num           185758 non-null  int64  
 7   winner_id           185758 non-null  int64  
 8   winner_name         185758 non-null  object 
 9   winner_hand         185758 non-null  object 
 10  winner_ht           185758 non-null  float64
 11  winner_ioc          185758 non-null  object 
 12  winner_age          185518 non-null  float64
 13  loser_id            185758 non-null  int64  
 14  loser_name          185758 non-null  object 
 15  loser_hand          185758 non-nul

# Geography

In [133]:
#e' necessario installare la libreria countryinfo
#pip install countryinfo
from countryinfo import CountryInfo

In [134]:
c= set(tennis['winner_ioc'].append(tennis['loser_ioc']))

In [135]:
g= pd.read_csv('data/countries.csv')

In [136]:
g

,country_code,country_name,continent
0,JPN,Japan,Asia
1,RUS,Russia,Europe
2,FRA,France,Europe
3,AUS,Australia,Oceania
4,CAN,Canada,America
...,...,...,...
119,AND,Andorra,Europe
120,DEU,Germany,Europe
121,NLD,Netherlands,Europe
122,NGA,Nigeria,Africa


In [137]:
g['country_code']

0      JPN
1      RUS
2      FRA
3      AUS
4      CAN
      ... 
119    AND
120    DEU
121    NLD
122    NGA
123    GRC
Name: country_code, Length: 124, dtype: object

Ci sono 154 distinti codici IOC in tennis.csv, invece in 'country.csv' ne abbiamo solo 124

In [138]:
len(c)

154

In [139]:
len(g['country_code'])

124

## Riempimento dei missing value in country_name

Utilizziamo quindi un dataset esterno contenente i codici IOC (International Olympic Committee) per riempire i codici mancanti mostrati sotto

In [140]:
missing=list(c - set(g['country_code']))

In [141]:
missing

['QAT',
 'ERI',
 'BOT',
 'CRI',
 'JOR',
 'AHO',
 'LBN',
 'GHA',
 'LVA',
 'SMR',
 'BEN',
 'JAM',
 'ITF',
 'SAU',
 'TTO',
 'COD',
 'BAN',
 'UAE',
 'ZAM',
 'AZE',
 'MRN',
 'TKM',
 'BGR',
 'BRN',
 'SYR',
 'VIN',
 'ANT',
 'BER',
 'LBA',
 'TOG']

In [142]:
g_new= g.append(pd.DataFrame(dict({'country_code': missing}))).reset_index(drop=True)

In [143]:
g_new

,country_code,country_name,continent
0,JPN,Japan,Asia
1,RUS,Russia,Europe
2,FRA,France,Europe
3,AUS,Australia,Oceania
4,CAN,Canada,America
...,...,...,...
149,VIN,NaN,NaN
150,ANT,NaN,NaN
151,BER,NaN,NaN
152,LBA,NaN,NaN


In [144]:
g_new['country_code']

0      JPN
1      RUS
2      FRA
3      AUS
4      CAN
      ... 
149    VIN
150    ANT
151    BER
152    LBA
153    TOG
Name: country_code, Length: 154, dtype: object

In [145]:
ioc= pd.read_csv('data/IOC_codes.csv')

In [146]:
ioc

,Country,NOC,ISO code
0,Afghanistan,AFG,AF
1,Albania,ALB,AL
2,Algeria,ALG,DZ
3,American Samoa*,ASA,AS
4,Andorra,AND,AD
...,...,...,...
196,Vietnam,VIE,VN
197,Virgin Islands*,ISV,VI
198,Yemen,YEM,YE
199,Zambia,ZAM,ZM


In [147]:
g_new.isnull().any(axis=1)

0      False
1      False
2      False
3      False
4      False
       ...  
149     True
150     True
151     True
152     True
153     True
Length: 154, dtype: bool

In [148]:
unknown=[]
for index,row in g_new[g_new.isnull().any(axis=1)].iterrows():
    try:
        print(row['country_code'])
        g_new.at[index,'country_name']= ioc[ioc['NOC']==row['country_code']].iloc[0].at['Country']
    except:
        unknown.append(row['country_code'])
        print('not found')

QAT
ERI
BOT
CRI
not found
JOR
AHO
LBN
not found
GHA
LVA
not found
SMR
BEN
JAM
ITF
not found
SAU
not found
TTO
not found
COD
BAN
UAE
ZAM
AZE
MRN
not found
TKM
BGR
not found
BRN
SYR
VIN
ANT
BER
LBA
TOG


In [149]:
g_new[g_new['country_name'].isnull()]

,country_code,country_name,continent
127,CRI,NaN,NaN
130,LBN,NaN,NaN
132,LVA,NaN,NaN
136,ITF,NaN,NaN
137,SAU,NaN,NaN
138,TTO,NaN,NaN
144,MRN,NaN,NaN
146,BGR,NaN,NaN


I restanti sono stati estrapolati dalla lista presente su wikipedia al link https://en.wikipedia.org/wiki/List_of_IOC_country_codes

In [150]:
index_LBN=g_new.index[g_new['country_code'] == 'LBN']
index_CRI=g_new.index[g_new['country_code'] == 'CRI']
index_BGR=g_new.index[g_new['country_code'] == 'BGR']
index_SAU=g_new.index[g_new['country_code'] == 'SAU']
index_TTO=g_new.index[g_new['country_code'] == 'TTO']
index_LVA=g_new.index[g_new['country_code'] == 'LVA']


In [151]:

g_new.at[index_TTO,'country_name']='Trinidad e Tobago'
g_new.at[index_BGR,'country_name']='Bulgaria'
g_new.at[index_LBN,'country_name']='Lebanon'
g_new.at[index_CRI,'country_name']='Costa Rica'
g_new.at[index_SAU,'country_name']='Saudi Arabia'
g_new.at[index_LVA,'country_name']='Latvia'

In [152]:
g_new[g_new['country_name'].isnull()]

,country_code,country_name,continent
136,ITF,NaN,NaN
144,MRN,NaN,NaN


Non è stato trovato nessun match con MRN, perciò lo si è scoperto cercando il nome alle tennista a cui è assegnato

In [153]:
tennis[(tennis['winner_ioc']=='MRN') | (tennis['loser_ioc']=='MRN')]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
52835,2016-W-WITF-FRA-01A-2016_10,Fort de France Martinique $10K,Hard,15,10,20160111,9,206091,Yolande Leacock,R,172.0,TRI,25.004791,214249,Angela Charles Alfred,U,172.0,MRN,16.350445,7-5 6-1,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1188.0,3.0,825.0,142.0,1442,293875.10
122845,2018-W-WITF-FRA-01A-2018_15,Fort de France Martinique $15K,Hard,30,15,20180108,5,201767,Sylvia Schenck,U,172.0,USA,53.949350,220990,Dieuvela Gaspard,R,172.0,MRN,16.837782,6-1 6-1,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,825.0,142.0,825.0,142.0,2737,536533.77
142053,2019-W-ITF-FRA-02A-2019_15,W15 Fort de France Martinique,Hard,32,15,20190107,12,215431,Emily Appleton,U,172.0,GBR,19.351129,220990,Dieuvela Gaspard,R,172.0,MRN,17.834360,6-1 6-0,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,740.0,11.0,825.0,142.0,3319,627368.84


In [154]:
g_new.at[g_new.index[g_new['country_code'] == 'MRN'],'country_name']='Martinique'

In [155]:
g_new[g_new['country_name'].isnull()]

,country_code,country_name,continent
136,ITF,NaN,NaN


Per quest'ultimo si è scoperto si tratti solamente di un typo si trattava infatti di un tennista italiano

In [156]:
tennis[(tennis['winner_ioc']=='ITF') | (tennis['loser_ioc']=='ITF')]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
72194,2016-7021_C,Mestre CH,Clay,32,C,20160516,13,106049,Ivan Sabanov,R,185.0,CRO,23.723477,206839,Filippo Piranomonte,U,185.0,ITF,14.409309,6-3 6-3,3,Q1,NaN,1.0,6.0,56.0,39.0,28.0,9.0,9.0,5.0,6.0,0.0,2.0,54.0,35.0,19.0,11.0,9.0,7.0,11.0,1149.0,8.0,825.0,142.0,4277,1144423.86


In [157]:
g_new= g_new.drop(g_new[g_new['country_name'].isnull()].index, axis=0)

In [158]:
g_new.reset_index(drop=True, inplace=True)

In [159]:
g_new

,country_code,country_name,continent
0,JPN,Japan,Asia
1,RUS,Russia,Europe
2,FRA,France,Europe
3,AUS,Australia,Oceania
4,CAN,Canada,America
...,...,...,...
148,VIN,Saint Vincent and the Grenadines,NaN
149,ANT,Antigua and Barbuda,NaN
150,BER,Bermuda*,NaN
151,LBA,Libya,NaN


In [160]:
for index,row in tennis[(tennis['winner_ioc']=='ITF') | (tennis['loser_ioc']=='ITF')].iterrows():
    if row['winner_ioc']=='ITF':
        tennis.at[index,'winner_ioc']= 'ITA'
    if row['loser_ioc']=='ITF':
        tennis.at[index,'loser_ioc']= 'ITA'

## Riempimento missing value in continent

Per reperire il continente corrispondente e successivamente anche la lingua utilizzeremo la libreria CountryInfo

In [161]:
g_new[g_new['continent'].isnull()]

,country_code,country_name,continent
124,QAT,Qatar,NaN
125,ERI,Eritrea,NaN
126,BOT,Botswana,NaN
127,CRI,Costa Rica,NaN
128,JOR,Jordan,NaN
129,AHO,Netherlands Antilles*,NaN
130,LBN,Lebanon,NaN
131,GHA,Ghana,NaN
132,LVA,Latvia,NaN
133,SMR,San Marino,NaN


In [162]:
CountryInfo('Bahrain').region()

'Asia'

In [163]:
for index,row in g_new[g_new['continent'].isnull()].iterrows():
    country= CountryInfo(row['country_name'])
    try:
        g_new.at[index,'continent']=country.region()
    except:
        print(row['country_name']+' not found')

Netherlands Antilles* not found
Trinidad e Tobago not found
Congo, Dem Rep not found
Bermuda* not found


Per i paesi senza match procediamo sostituendoli manualmente

In [164]:
g_new[g_new['continent'].isnull()]

,country_code,country_name,continent
129,AHO,Netherlands Antilles*,NaN
137,TTO,Trinidad e Tobago,NaN
138,COD,"Congo, Dem Rep",NaN
150,BER,Bermuda*,NaN


In [165]:

g_new.at[g_new.index[g_new['country_code'] == 'COD'],'continent']='Africa'
g_new.at[g_new.index[g_new['country_code'] == 'TTO'],'continent']='America'
g_new.at[g_new.index[g_new['country_code'] == 'AHO'],'continent']='America'
g_new.at[g_new.index[g_new['country_code'] == 'BER'],'continent']='America'

In [166]:
g_new['continent'].replace('Americas','America', inplace=True, regex=True)

In [167]:
g_new[124:160]

,country_code,country_name,continent
124,QAT,Qatar,Asia
125,ERI,Eritrea,Africa
126,BOT,Botswana,Africa
127,CRI,Costa Rica,America
128,JOR,Jordan,Asia
129,AHO,Netherlands Antilles*,America
130,LBN,Lebanon,Asia
131,GHA,Ghana,Africa
132,LVA,Latvia,Europe
133,SMR,San Marino,Europe


## Aggiunta attributo language 

Qui, sempre utilizzando la libreria andiamo ad ottenere la prima lingua (tra quelle ufficiali)

In [168]:
CountryInfo('Italy').languages()

['it']

In [169]:
for index, row in g_new.iterrows():
    c= CountryInfo(row['country_name'])
    try:
        g_new.at[index,'language']=c.languages()[0]
    except: 
        print(row['country_name']+' not found')

New Zeland not found
Urugay not found
Montenegro not found
Bahamas not found
North Macedonia not found
Malaysia not found
Kosovo not found
Unknown not found
Andorra not found
Netherlands Antilles* not found
Trinidad e Tobago not found
Congo, Dem Rep not found
Bermuda* not found


In [170]:
g_new[g_new['language'].isnull()]

,country_code,country_name,continent,language
27,NZL,New Zeland,Oceania,NaN
34,URU,Urugay,America,NaN
76,MNE,Montenegro,Europe,NaN
82,BAH,Bahamas,America,NaN
87,MKD,North Macedonia,Europe,NaN
89,MAS,Malaysia,Asia,NaN
98,UNK,Kosovo,Europe,NaN
107,POC,Unknown,Unknown,NaN
119,AND,Andorra,Europe,NaN
129,AHO,Netherlands Antilles*,America,NaN


In [171]:
g_new.at[g_new.index[g_new['country_code'] == 'POC'],'country_name']='Philippines'
g_new.at[g_new.index[g_new['country_code'] == 'POC'],'continent']='Asia'
g_new.at[g_new.index[g_new['country_code'] == 'POC'],'language']='fil'

In [172]:
g_new

,country_code,country_name,continent,language
0,JPN,Japan,Asia,ja
1,RUS,Russia,Europe,ru
2,FRA,France,Europe,fr
3,AUS,Australia,Oceania,en
4,CAN,Canada,America,en
...,...,...,...,...
148,VIN,Saint Vincent and the Grenadines,America,en
149,ANT,Antigua and Barbuda,America,en
150,BER,Bermuda*,America,NaN
151,LBA,Libya,Africa,ar


In [173]:
g_new[g_new['language'].isnull()]

,country_code,country_name,continent,language
27,NZL,New Zeland,Oceania,NaN
34,URU,Urugay,America,NaN
76,MNE,Montenegro,Europe,NaN
82,BAH,Bahamas,America,NaN
87,MKD,North Macedonia,Europe,NaN
89,MAS,Malaysia,Asia,NaN
98,UNK,Kosovo,Europe,NaN
119,AND,Andorra,Europe,NaN
129,AHO,Netherlands Antilles*,America,NaN
137,TTO,Trinidad e Tobago,America,NaN


In [174]:
g_new.at[g_new.index[g_new['country_code'] == 'NZL'],'country_name']='New Zealand'
g_new.at[g_new.index[g_new['country_code'] == 'URU'],'country_name']='Uruguay'

In [175]:
for index, row in g_new[g_new['language'].isnull()].iterrows():
    c= CountryInfo(row['country_name'])
    try:
        g_new.at[index,'language']=c.languages()[0]
    except: 
        print(row['country_name']+' not found')

Montenegro not found
Bahamas not found
North Macedonia not found
Malaysia not found
Kosovo not found
Andorra not found
Netherlands Antilles* not found
Trinidad e Tobago not found
Congo, Dem Rep not found
Bermuda* not found


In [176]:
g_new[g_new['language'].isnull()]

,country_code,country_name,continent,language
76,MNE,Montenegro,Europe,NaN
82,BAH,Bahamas,America,NaN
87,MKD,North Macedonia,Europe,NaN
89,MAS,Malaysia,Asia,NaN
98,UNK,Kosovo,Europe,NaN
119,AND,Andorra,Europe,NaN
129,AHO,Netherlands Antilles*,America,NaN
137,TTO,Trinidad e Tobago,America,NaN
138,COD,"Congo, Dem Rep",Africa,NaN
150,BER,Bermuda*,America,NaN


Per i paesi senza match andiamo ad inserirla manualmente

In [177]:
g_new.at[g_new.index[g_new['country_code'] == 'MNE'],'language']='sr'
g_new.at[g_new.index[g_new['country_code'] == 'BAH'],'language']='en'
g_new.at[g_new.index[g_new['country_code'] == 'MKD'],'language']='mk'
g_new.at[g_new.index[g_new['country_code'] == 'MAS'],'language']='ms'
g_new.at[g_new.index[g_new['country_code'] == 'UNK'],'language']='sq'
g_new.at[g_new.index[g_new['country_code'] == 'AND'],'language']='ca'
g_new.at[g_new.index[g_new['country_code'] == 'BER'],'language']='en'
g_new.at[g_new.index[g_new['country_code'] == 'AHO'],'language']='fr'
g_new.at[g_new.index[g_new['country_code'] == 'TTO'],'language']='en'
g_new.at[g_new.index[g_new['country_code'] == 'COD'],'language']='en'

In [178]:
g_new

,country_code,country_name,continent,language
0,JPN,Japan,Asia,ja
1,RUS,Russia,Europe,ru
2,FRA,France,Europe,fr
3,AUS,Australia,Oceania,en
4,CAN,Canada,America,en
...,...,...,...,...
148,VIN,Saint Vincent and the Grenadines,America,en
149,ANT,Antigua and Barbuda,America,en
150,BER,Bermuda*,America,en
151,LBA,Libya,Africa,ar


In [179]:
g_new.isnull().any(axis=0)

country_code    False
country_name    False
continent       False
language        False
dtype: bool

In [180]:
g_new.drop('country_name', axis=1, inplace=True)

In [181]:
g_new.rename(columns={'country_code':'country_ioc'}, inplace=True)

In [182]:
g_new.to_csv('table/countries_new.csv',index=False)

In [183]:
tennis.to_csv('data/tennis_new.csv', index=False)
male.to_csv('data/male_new.csv', index=False)
female.to_csv('data/female_new.csv', index=False)